"""
BreakPhaseDataProcessor module processes data from the experiment 
processing participants physiological data xlxs files and 
pulling break lenght data from virtual environment dataset to merge it for analysis
"""

# init (setup of the environment):

In [1]:
def setupFunction(setupFull,suppressWarnings):
    
    if setupFull:
        #!pip install pathlib
        #!pip install -U setuptools pip
        #!pip uninstall cupy
        #!git clone --recursive https://github.com/cupy/cupy.git
        #!cd cupy
        #!pip cupy install --no-cache-dir
        
        # Make sure to run notebook with admin privilages when installing packages

        !pip install https://github.com/paulvangentcom/heartrate_analysis_python/archive/master.zip

        #update pip if using git+
        !pip install --upgrade pip

        !pip install git+https://github.com/paulvangentcom/heartrate_analysis_python.git

        !pip3 install --upgrade pandas --user

    # for graphing library: 
        !pip install bokeh --user
        !pip install numpy --no-index --trusted-host=None --find-links=./ --user
        !pip install scipy --no-index --trusted-host=None --find-links=./ --user
        !pip install scikit-learn --no-index --trusted-host=None --find-links=./ --user
        !pip install cvxopt --no-index --trusted-host=None --find-links=./ --user
        !pip install kiwisolver --no-index --trusted-host=None --find-links=./ --user
        !pip install cycler --no-index --trusted-host=None --find-links=./ --user
        !pip install dateutils --user
        !pip install pyparsing --user
        !pip install matplotlib --no-index --trusted-host=None --find-links=./ --user
        !pip install PyQt5 --user
        !pip install pylint --user
        !pip install spyder --user
        !pip install https://github.com/neuropsychology/NeuroKit.py/zipball/master --user
        !pip install https://github.com/neuropsychology/Neuropsydia.py/zipball/master --user
        !pip install gcloud --user
        !pip install --upgrade setuptools --user
    #PROBLEMS WITH LIB BELOW
        #pip install https://github.com/lciti/cvxEDA/blob/792844420df7d83c2e061cc8a7bb8ca24d4c29b5/src/cvxEDA.py --user
        #ERROR: Cannot determine archive format
        #!pip install https://github.com/lciti/cvxEDA/archive/master.zip --user
        
        !pip install https://github.com/HIIT/Ledapy/archive/master.zip --user
    
    if(suppressWarnings):
        warnings.filterwarnings('ignore')

In [2]:
import sys
import time
import datetime
import os
import warnings

import numpy as np
import pandas as pd

import heartpy as hp

from bokeh.plotting import figure, output_file, save
import matplotlib.pyplot as plt

import cvxopt as cv
import cvxopt.solvers



setupFull = False
suppressWarnings = False # tons of errors from data being defragmented, but most likley due to last update
setupFunction(setupFull, suppressWarnings)

For multiprocessing:

In [3]:
import threading # used in first fundamental approach
import concurrent.futures #used for more optimized multithreading 

In [4]:
"""
Function cvxEDA embeded here due to an error: 
File "setup.py" not found for legacy project https://github.com/lciti/cvxEDA/archive/master.zip
______________________________________________________________________________
 File:                         cvxEDA.py
 Last revised:                 07 Nov 2015 r69
 ______________________________________________________________________________
 Copyright (C) 2014-2015 Luca Citi, Alberto Greco
 
 This program is free software; you can redistribute it and/or modify it under
 the terms of the GNU General Public License as published by the Free Software
 Foundation; either version 3 of the License, or (at your option) any later
 version.
 
 This program is distributed in the hope that it will be useful, but WITHOUT
 ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
 FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.
 
 You may contact the author by e-mail (lciti@ieee.org).
 ______________________________________________________________________________
 This method was first proposed in:
 A Greco, G Valenza, A Lanata, EP Scilingo, and L Citi
 "cvxEDA: a Convex Optimization Approach to Electrodermal Activity Processing"
 IEEE Transactions on Biomedical Engineering, 2015
 DOI: 10.1109/TBME.2015.2474131
 If you use this program in support of published research, please include a
 citation of the reference above. If you use this code in a software package,
 please explicitly inform the end users of this copyright notice and ask them
 to cite the reference above in their published research.
 ______________________________________________________________________________
"""

import numpy as np
"""

IMPORTANT BELOW!!!!!!!!!!!!!!!!!!!!!

"""
# can try to use CuPy instead of numpy to push the parallel processing of this data to GPU
#import CuPy
import cvxopt as cv
import cvxopt.solvers

def cvxEDA(y, delta, tau0=2., tau1=0.7, delta_knot=10., alpha=8e-4, gamma=1e-2,
           solver=None, options={'reltol':1e-9}):
    """CVXEDA Convex optimization approach to electrodermal activity processing
    This function implements the cvxEDA algorithm described in "cvxEDA: a
    Convex Optimization Approach to Electrodermal Activity Processing"
    (http://dx.doi.org/10.1109/TBME.2015.2474131, also available from the
    authors' homepages).
    Arguments:
       y: observed EDA signal (we recommend normalizing it: y = zscore(y))
       delta: sampling interval (in seconds) of y
       tau0: slow time constant of the Bateman function
       tau1: fast time constant of the Bateman function
       delta_knot: time between knots of the tonic spline function
       alpha: penalization for the sparse SMNA driver
       gamma: penalization for the tonic spline coefficients
       solver: sparse QP solver to be used, see cvxopt.solvers.qp
       options: solver options, see:
                http://cvxopt.org/userguide/coneprog.html#algorithm-parameters
    Returns (see paper for details):
       r: phasic component
       p: sparse SMNA driver of phasic component
       t: tonic component
       l: coefficients of tonic spline
       d: offset and slope of the linear drift term
       e: model residuals
       obj: value of objective function being minimized (eq 15 of paper)
    """

    n = len(y)
    y = cv.matrix(y)

    # bateman ARMA model
    a1 = 1./min(tau1, tau0) # a1 > a0
    a0 = 1./max(tau1, tau0)
    ar = np.array([(a1*delta + 2.) * (a0*delta + 2.), 2.*a1*a0*delta**2 - 8.,
        (a1*delta - 2.) * (a0*delta - 2.)]) / ((a1 - a0) * delta**2)
    ma = np.array([1., 2., 1.])

    # matrices for ARMA model
    i = np.arange(2, n)
    A = cv.spmatrix(np.tile(ar, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))
    M = cv.spmatrix(np.tile(ma, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))

    # spline
    delta_knot_s = int(round(delta_knot / delta))
    spl = np.r_[np.arange(1.,delta_knot_s), np.arange(delta_knot_s, 0., -1.)] # order 1
    spl = np.convolve(spl, spl, 'full')
    spl /= max(spl)
    # matrix of spline regressors
    i = np.c_[np.arange(-(len(spl)//2), (len(spl)+1)//2)] + np.r_[np.arange(0, n, delta_knot_s)]
    nB = i.shape[1]
    j = np.tile(np.arange(nB), (len(spl),1))
    p = np.tile(spl, (nB,1)).T
    valid = (i >= 0) & (i < n)
    B = cv.spmatrix(p[valid], i[valid], j[valid])

    # trend
    C = cv.matrix(np.c_[np.ones(n), np.arange(1., n+1.)/n])
    nC = C.size[1]

    # Solve the problem:
    # .5*(M*q + B*l + C*d - y)^2 + alpha*sum(A,1)*p + .5*gamma*l'*l
    # s.t. A*q >= 0

    old_options = cv.solvers.options.copy()
    cv.solvers.options.clear()
    cv.solvers.options.update(options)
    if solver == 'conelp':
        # Use conelp
        z = lambda m,n: cv.spmatrix([],[],[],(m,n))
        G = cv.sparse([[-A,z(2,n),M,z(nB+2,n)],[z(n+2,nC),C,z(nB+2,nC)],
                    [z(n,1),-1,1,z(n+nB+2,1)],[z(2*n+2,1),-1,1,z(nB,1)],
                    [z(n+2,nB),B,z(2,nB),cv.spmatrix(1.0, range(nB), range(nB))]])
        h = cv.matrix([z(n,1),.5,.5,y,.5,.5,z(nB,1)])
        c = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T,z(nC,1),1,gamma,z(nB,1)])
        res = cv.solvers.conelp(c, G, h, dims={'l':n,'q':[n+2,nB+2],'s':[]})
        obj = res['primal objective']
    else:
        # Use qp
        Mt, Ct, Bt = M.T, C.T, B.T
        H = cv.sparse([[Mt*M, Ct*M, Bt*M], [Mt*C, Ct*C, Bt*C], 
                    [Mt*B, Ct*B, Bt*B+gamma*cv.spmatrix(1.0, range(nB), range(nB))]])
        f = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T - Mt*y,  -(Ct*y), -(Bt*y)])
        res = cv.solvers.qp(H, f, cv.spmatrix(-A.V, A.I, A.J, (n,len(f))),
                            cv.matrix(0., (n,1)), solver=solver)
        obj = res['primal objective'] + .5 * (y.T * y)
    
    
    cv.solvers.options.clear()
    cv.solvers.options.update(old_options)

    l = res['x'][-nB:]
    d = res['x'][n:n+nC]
    t = B*l + C*d
    q = res['x'][:n]
    p = A * q
    r = M * q
    e = y - r - t

    return (np.array(a).ravel() for a in (r, p, t, l, d, e, obj))
    """
    BELOW COMMENTED OUT TO ENABLE OPTIONS
    ????? Double check it later
    ????? seems to be clearing and reseting if in the loop?
    ...new approach pass dictionary of solver parameters as an option...in test 
    """

In [5]:
def settingUpPaths():
    os.chdir(os.path.abspath(os.curdir))

In [6]:
settingUpPaths()

# Functions:

## 1 : Environment (simulation) data:

In [7]:
def pullEnvironmentData(ID, GR): #participantID,participantGR
    #First_VR_Experiment_Files/
    pathToEnvironmentData = 'RawData/Environment Parameters/'
    environmentDataPath = os.path.join(os.getcwd(), pathToEnvironmentData)
    filePath = 'Sample_Number-' + str(ID) + '_Group_Number-' + str(GR) + '.csv'    
    pathToEnvironmentFile = os.path.join(environmentDataPath, filePath)
    environment_data = pd.read_csv(pathToEnvironmentFile , delimiter=',', dtype=None, parse_dates = True, skiprows = 1, infer_datetime_format = False, index_col=None, header = None,low_memory = False)
    environmentColNames = ["timestamp","Ticks","Frame_count","Phase","Time_in_Phase","Real_Time_in_Phase","Trial_Number","Trial_Time","Period_Type","Time_in_Period","Fidelity","Model","FOV","Saturation","Resolution","Pixel_count","Thread_sleep","FPS_real","Audio_clip","STRANGE"]
    environment_data.columns = environmentColNames
    
    environment_data['timestamp'] = pd.to_datetime(environment_data['timestamp'], format="%d/%m/%Y %H:%M:%S.%f", errors='coerce')
    environment_data.set_index('timestamp')
   
    del environment_data['STRANGE']

    return environment_data

## 2 : Physiological (participant) data:

### 2.1 : pull physiological data from file:

In [8]:
def pullPhysioData(ID, GR):#participantID,participantGR
    #aadSize = pd.Timedelta(seconds=4)
    #sliceStart = experimentStart - pd.Timedelta(seconds=4) - padSize
    #sliceEnd = experimentEnd + padSize
    
    #First_VR_Experiment_Files/
    pathToPhysiologicalData = 'RawData/Participant biofeedback/'
    physiologicaDataPath = os.path.join(os.getcwd(), pathToPhysiologicalData)
    #print("physiologicaDataPath" + physiologicaDataPath)
    filePath = str(ID) + '-' + str(GR) + '.csv'    
    pathToPhysiologicaFile = os.path.join(physiologicaDataPath, filePath)
    #print("pathToPhysiologicaFile" + pathToPhysiologicaFile)
    
    data = pd.read_csv(pathToPhysiologicaFile,
                       delimiter='\t', 
                       dtype=None, 
                       parse_dates = True,
                       skiprows = 3, 
                       infer_datetime_format = False, 
                       index_col=None, 
                       header = None, 
                       low_memory = False)
    
    participantsColNames = ["timestamp",
                            "A15",
                            "A6",
                            "A7",
                            "GSR_Range",
                            "Skin_Conductance",
                            "Skin_Resistance",
                            "PPG_A13",
                            "PPG_IBI",
                            "PPG_to_HR_CONSENSYS",
                            "Pressure",
                            "Temperature",
                            "STRANGE"]
    
    data.columns = participantsColNames
    data.set_index('timestamp')
    data['timestamp'] = pd.to_datetime(data['timestamp'], infer_datetime_format  = True)
    data["TIME"] = data['timestamp']
    del data['STRANGE']
    return data

#### 2.1.1 Test HR signal quality

In [9]:
def qualityCheck(data):
    dataLen = len(data['PPG_to_HR_CONSENSYS'])
    ambigousMeasuresCount = 0
    for point in data['PPG_to_HR_CONSENSYS']:
        if point < 0:
            ambigousMeasuresCount += 1
        
    ambigousMeasuresCount = (1-((dataLen - ambigousMeasuresCount)/dataLen)) * 100
    if testModeGlobal:
        print(f"Percentage of ambigous HR data in the sample = {ambigousMeasuresCount} %")
    return ambigousMeasuresCount

### 2.2 : interpolate HR = -1 (NA) values -> append HR_CONSENSYS_interpolated column

In [10]:
def interpolateConsensysHRna(data, testMode = False):
    
    participant_data_HR_Interpolated = data['PPG_to_HR_CONSENSYS'].replace(-1, np.nan)
    
    if(np.isnan(participant_data_HR_Interpolated[0])):
        if(testMode):
            print("Consensys_HR : BACKFILL")
        firstValueIndex = pd.notnull(participant_data_HR_Interpolated).argmax()
        participant_data_HR_Interpolated[0] = participant_data_HR_Interpolated[firstValueIndex]
    
    participant_data_HR_Interpolated.interpolate(inplace=True)
    data['HR_CONSENSYS_interpolated'] = participant_data_HR_Interpolated
    
    """OLD
    participant_data_HR_Interpolated.interpolate(inplace=True)
    location = data.columns.get_loc("PPG_to_HR_CONSENSYS") + 1
    data.insert(loc=location, column='HR_CONSENSYS_interpolated', value=participant_data_HR_Interpolated)
    """
    
    participant_data_IBI_Interpolated = data['PPG_IBI'].replace(-1, np.nan)
        
    if(np.isnan(participant_data_IBI_Interpolated[0])):
        if(testMode):
            print("Consensys_IBI : BACKFILL")
        firstValueIndex = pd.notnull(participant_data_IBI_Interpolated).argmax()
        participant_data_IBI_Interpolated[0] = participant_data_IBI_Interpolated[firstValueIndex]
    
    participant_data_IBI_Interpolated.interpolate(inplace=True)
    data['IBI_CONSENSYS_interpolated'] = participant_data_IBI_Interpolated
    
    """OLD        
    participant_data_IBI_Interpolated.interpolate(inplace=True)
    location = data.columns.get_loc("PPG_IBI") + 1
    data.insert(loc=location, column='IBI_CONSENSYS_interpolated', value = participant_data_IBI_Interpolated)
    """
    
    return data

### 2.3 : add HRV and continous HR measures using HeartPy (hp)

In [11]:
def make_windows(data, sample_rate, windowsize=120, overlap=0, min_size=20):
    '''make_windows function comes from the heartpy module but in 1.6 it throws access error hence moved to local:
    
    function that slices data into windows for concurrent analysis.
    
    keyword arguments:
    - data: 1-dimensional numpy array containing heart rate sensor data
    - sample_rate: sample rate of the data stream in 'data'
    - windowsize: size of the window that is sliced
    - overlap: overlap between two adjacent windows: 0 <= float < 1.0
    - min_size: the minimum size for the last (partial) window to be included. Very short windows
                might not stable for peak fitting, especially when significant noise is present. 
                Slightly longer windows are likely stable but don't make much sense from a 
                signal analysis perspective.
    
    returns index tuples of windows
    '''
    ln = len(data)
    window = windowsize * sample_rate
    stepsize = (1 - overlap) * window
    start = 0
    end = window
    
    slices = []
    while end < len(data):
        slices.append((start, end))
        start += stepsize
        end += stepsize
    if (ln - start) / sample_rate >= min_size:
        slices.append((start, ln))
        
    return np.array(slices, dtype=np.int32)

In [12]:
def getPeriod(data, sampleRate, winSize, overLap):
    slicesArray = make_windows(data, sample_rate = sampleRate, windowsize = winSize, overlap = overLap)
    # if using make_windows from heartpy make sure to change to hp.meak_windows()
    period = slicesArray[1,0]
    return period

In [13]:
def getContDataFromHP(data, sampleRate, winSize, overLap, scale, period, testMode = False):
    
    if testMode:
        startTime0 = time.time()
        print("================TEST getContDataFromHP ===============" )
        print("type(data) : {}".format(type(data)))
        print("len(data) : {}".format(len(data)))
    
    """
    try:
        working_data, measures = hp.process(data, 
                                            sample_rate = sampleRate,
                                            #segment_width = winSize,
                                            windowsize = winSize,
                                            #segment_overlap =overLap, 
                                            # mode = "full", 
                                            # segment_min_size = -1, 
                                            # clipping_scale = scale, 
                                            # hampel_correct = True
                                            calc_freq = True,
                                            high_precision = True,
                                            clean_rr = True,# (bool) - if true, the RR_list is further cleaned with an outlier rejection pass default : False
                                            )
    except Exception as e:
        print("error_3 : HP.process()")
        print(e)
    
    #print("measures.get('bpm')")
    #print(measures.get('bpm'))
    
    #print("measures.items()")
    #print(measures.items())
    """    
    
    try:
        segmented_working_data, segmented_measures = hp.process_segmentwise(data, 
                                                                            sample_rate = sampleRate,
                                                                            segment_width = winSize,
                                                                            segment_overlap = overLap, 
                                                                            mode = "full", 
                                                                            segment_min_size = -1, 
                                                                            clipping_scale = scale, 
                                                                            hampel_correct = True)#, 
                                                                        #reject_segmentwise = True, bpmmin = 1, bpmmax = 5000)#, hampel_correct = True)
        
    except Exception as e:
        print("error_3 : HP -> process_segmentwise")
        print(e)
    
    hp_data = pd.DataFrame()
    
    for key, value in segmented_measures.items():   
        hp_data[key] = value
        
        if testMode:
            print(key)
            print(type(value))
            print(len(value))
    

    if testMode:
        print("--------hp_data after for key, value in segmented_measures.items()-------")
        print(hp_data.head(2))
        print(hp_data.tail(2))

        print("----------")
        print(f"type(hp_data['segment_indices'].iloc[0]) : {type(hp_data['segment_indices'].iloc[0])}")
        print("----------")
        print("hp_data['segment_indices'].iloc[0:3]")
        print(hp_data['segment_indices'].iloc[0:3])
        print("hp_data['segment_indices'].iloc[-3:]")
        print(hp_data['segment_indices'].iloc[-3:])
        print("----------")

        print("----------CONVERSION---------------")
    
        startTime99 = time.time()
    
    hp_data['segment_indices'] = [sum(indices) // 2 for indices in hp_data['segment_indices']]
    
    if testMode:
        print(f"Time to average out : {time.time() - startTime99}")

        print(f"type(hp_data['segment_indices'].iloc[0]) : {type(hp_data['segment_indices'].iloc[0])}")
        print("----------")
        print("hp_data['segment_indices'].iloc[0:3]")
        print(hp_data['segment_indices'].iloc[0:3])
        print("hp_data['segment_indices'].iloc[-3:]")
        print(hp_data['segment_indices'].iloc[-3:])
        print("----------")
    
    NewDF = pd.DataFrame(np.nan , index =  range(0, len(data)) , columns = hp_data.columns)
    
    lastInd = len(hp_data)-1
    
    if testMode:
        startTime = time.time()
        print(NewDF.head(2))
        print(NewDF.tail(2))
        
        print(f"Last index : {lastInd}")
    
        timeForInterpolation = []
    
    for column in hp_data:
        if testMode:
            print("column : {}".format(column))
        
        #setting up first and last index is not nescessery, will be better to interpolate?
        """
        print("hp_data[column].iloc[0] : {}".format(hp_data[column].iloc[0]))
        NewDF[column].iloc[0] = hp_data[column].iloc[0]
        print("NewDF[column].iloc[0] : {}".format(NewDF[column].iloc[0]))
        
        print("hp_data[column].iloc[-1] : {}".format(hp_data[column].iloc[-1]))
        NewDF[column].iloc[len(hp_data)-1] = hp_data[column].iloc[-1]
        print("NewDF[column].iloc[] : {}".format(hp_data[column].iloc[-1]))
        """
        
        i = 0        
        for item in hp_data[column]:
            if testMode:
                print("item : {}".format(item))
            
            NewDF[column].iloc[hp_data['segment_indices'].iloc[i]] = item
            i += 1
            
        if testMode:
            startTime = time.time()
        
        NewDF[column].interpolate(method='cubic', order=2, inplace = True)#(method='polynomial', order=2, inplace = True)
        #NewDF[column].interpolate(method='linear', inplace = True)
        
        if testMode:
            timeForInterpolation.append(time.time() - startTime)
    
        if testMode:
            print(f"interpolations took : {sum(timeForInterpolation)/len(timeForInterpolation)}")
    
    firstInd = hp_data['segment_indices'].iloc[0]
    lastInd = hp_data['segment_indices'].iloc[-1]
    
    if testMode:
        print("---------interpolated-----------")
        print("---------------------NewDF.head(2)")
        print(NewDF.head(2))
        print("NewDF[firstInd - 3 : firstInd + 3]")
        print(NewDF[firstInd - 3 : firstInd + 3])
        print("NewDF[lastInd - 3 : lastInd + 3]")
        print(NewDF[lastInd - 3 : lastInd + 3])
        print("---------------------NewDF.tail(2)")
        print(NewDF.tail(2))
        print("---------interpolated END-----------")
    
    hp_data = NewDF
    
    if testMode:
        print("---------------------hp_data.head(2)")
        print(hp_data.head(2))
        print("---------------------hp_data.tail(2)")
        print(hp_data.tail(2))

        print("PRESTEST TOOK : {}".format(time.time() - startTime))
        print("=======PRETEST END=========")
    
    # Interpolate NA:
    
    # HR:
    hp_HR_Interpolated = hp_data['bpm'].replace(-1, np.nan)
    hp_HR_Interpolated[hp_HR_Interpolated>200] = np.nan
    hp_HR_Interpolated[hp_HR_Interpolated<30] = np.nan
    
    if(np.isnan(hp_HR_Interpolated[0])):
        if testMode:
            print("hp_HR_Interpolated : BACKFILL")
        firstValueIndex = pd.notnull(hp_HR_Interpolated).argmax()
        hp_HR_Interpolated[0] = hp_HR_Interpolated[firstValueIndex]

    hp_HR_Interpolated.interpolate(inplace=True)
    hp_data['HR_HeartPy_interpolated'] = hp_HR_Interpolated
    
    # IBI:
    hp_IBI_Interpolated = hp_data['ibi'].replace(-1, np.nan)
    
    if(np.isnan(hp_IBI_Interpolated[0])):
        if testMode:
            print("hp_IBI_Interpolated : BACKFILL")
        firstValueIndex = pd.notnull(hp_IBI_Interpolated).argmax()
        hp_IBI_Interpolated[0] = hp_IBI_Interpolated[firstValueIndex]

    hp_IBI_Interpolated.interpolate(inplace=True)
    hp_data['IBI_HeartPy_interpolated'] = hp_IBI_Interpolated
    
    # sdnn:
    hp_SDNN_Interpolated = hp_data['sdnn'].replace(-1, np.nan)
    
    if(np.isnan(hp_SDNN_Interpolated[0])):
        if testMode:
            print("hp_SDNN_Interpolated : BACKFILL")
        firstValueIndex = pd.notnull(hp_SDNN_Interpolated).argmax()
        hp_SDNN_Interpolated[0] = hp_SDNN_Interpolated[firstValueIndex]
    
    hp_IBI_Interpolated.interpolate(inplace=True)
    hp_data['SDNN_HeartPy_interpolated'] = hp_IBI_Interpolated
    
    # sdsd:
    hp_SDSD_Interpolated = hp_data['sdsd'].replace(-1, np.nan)
    
    if(np.isnan(hp_SDSD_Interpolated[0])):
        if testMode:
            print("hp_SDSD_Interpolated : BACKFILL")
        firstValueIndex = pd.notnull(hp_SDSD_Interpolated).argmax()
        hp_SDSD_Interpolated[0] = hp_SDSD_Interpolated[firstValueIndex]
    
    hp_SDSD_Interpolated.interpolate(inplace=True)
    hp_data['SDSD_HeartPy_interpolated'] = hp_SDSD_Interpolated
    
    # rmssd:
    hp_RMSSD_Interpolated = hp_data['rmssd'].replace(-1, np.nan)
    
    if(np.isnan(hp_RMSSD_Interpolated[0])):
        if testMode:
            print("hp_RMSSD_Interpolated : BACKFILL")
        firstValueIndex = pd.notnull(hp_RMSSD_Interpolated).argmax()
        hp_RMSSD_Interpolated[0] = hp_RMSSD_Interpolated[firstValueIndex]
    
    hp_RMSSD_Interpolated.interpolate(inplace=True)
    hp_data['RMSSD_HeartPy_interpolated'] = hp_RMSSD_Interpolated
    
    # PNN-20:
    
    hp_PNN20_Interpolated = hp_data['pnn20'].replace(-1, np.nan)
    
    if(np.isnan(hp_PNN20_Interpolated[0])):
        if testMode:
            print("hp_PNN20_Interpolated : BACKFILL")
        firstValueIndex = pd.notnull(hp_PNN20_Interpolated).argmax()
        hp_PNN20_Interpolated[0] = hp_PNN20_Interpolated[firstValueIndex]
    
    hp_PNN20_Interpolated.interpolate(inplace=True)
    hp_data['pNN20_HeartPy_interpolated'] = hp_PNN20_Interpolated
    
    # PNN-50:
    hp_PNN50_Interpolated = hp_data['pnn50'].replace(-1, np.nan)
    
    if(np.isnan(hp_PNN50_Interpolated[0])):
        if testMode:
            print("hp_PNN20_Interpolated : BACKFILL")
        firstValueIndex = pd.notnull(hp_PNN50_Interpolated).argmax()
        hp_PNN50_Interpolated[0] = hp_PNN50_Interpolated[firstValueIndex]
    
    hp_PNN50_Interpolated.interpolate(inplace=True)
    hp_data['pNN50_HeartPy_interpolated'] = hp_PNN50_Interpolated
    
    if testMode:
        print("TEST END getContDataFromHP ========{}".format(time.time() - startTime0))

        print("getContDataFromHP after all interpolations")
        print(hp_data.head())
        print("hp_data[firstInd - 3 : firstInd + 3]")
        print(hp_data[firstInd - 3 : firstInd + 3])
        print("hp_data[lastInd - 3 : lastInd + 3]")
        print(hp_data[lastInd - 3 : lastInd + 3])
        print(hp_data.tail())

    return hp_data

#### 2.3.2 : Save peak analysis plot

In [14]:
def savePlot(data, sampleRate, winSize, scale):
    working_data, measures = hp.process(data, sample_rate= sampleRate, windowsize= winSize, clipping_scale = scale, reject_segmentwise = False, bpmmin = 1, bpmmax = 5000)#, hampel_correct = True)
    plot_object = hp.plotter(working_data, measures, show = False)
    graphPath = "Graphs/"+ participantID + "_" + str(participantGR) + "-HP_peaks.svg";
    plot_object.savefig(os.path.join(os.getcwd(),graphPath), dpi = 300)

#### 2.3.3 : Save HR plot figure

In [15]:
def saveHRplot(ID, GR, HP_data, Consensys_data):   
    graphPath = "Graphs\/"+ str(ID) + "_" + str(GR) + "-HR.html";
    output_file(graphPath)
    
    objectToPlot = figure(
        #y_range = (0,1),
        #x_range = (0,1),
        title = 'Heart Rate data - ID: ' + str(ID) + ' GR: ' + str(GR),
        x_axis_label = 'Index[i]',
        y_axis_label = 'HR[BPM]',
        plot_width = 1600,
        #plot_height = 900,
        sizing_mode='scale_width'
    )
    
    objectToPlot.line(HP_data.index.tolist(), HP_data['HR_HeartPy_interpolated'], 
                      legend_label='HP_HR-interpolated', line_color="red", line_width = 1)
    objectToPlot.line(HP_data.index.tolist(), HP_data['bpm'], 
                      legend_label='HeartPy_HR', line_color="blue", line_width = 4,
                      line_dash = 'dotted', line_alpha = .7)
    objectToPlot.line(Consensys_data.index.tolist(), Consensys_data['PPG_to_HR_CONSENSYS'], 
                      legend_label='Consensys_HR', line_color="green", line_width = 4,
                      line_dash = 'dotted', line_alpha = .7)
    objectToPlot.line(Consensys_data.index.tolist(), Consensys_data['HR_CONSENSYS_interpolated'], 
                      legend_label='Consensys_HR-interpolated', line_color="green", line_width = 1)
    
    objectToPlot.background_fill_color = "beige"
    objectToPlot.background_fill_alpha = 0.5
    save(objectToPlot)

### 2.4 : ***Merge Consensys and HeartPy data***

In [16]:
def mergeData(data1, data2):
    mergedData = pd.concat([data1, data2], axis= 1)
    return mergedData

### 2.5 : ***standardize data (Z-score)***

In [17]:
def standardizeData(data):
    data['Skin_Conductance_STD'] = (data['Skin_Conductance'] - data['Skin_Conductance'].mean())/data['Skin_Conductance'].std()
    data['Skin_Resistance_STD'] = (data['Skin_Resistance'] - data['Skin_Resistance'].mean())/data['Skin_Resistance'].std()
    data['HR_CONSENSYS_STD'] = (data['HR_CONSENSYS_interpolated'] - data['HR_CONSENSYS_interpolated'].mean())/data['HR_CONSENSYS_interpolated'].std()
    data['HR_HP_STD'] = (data['HR_HeartPy_interpolated'] - data['HR_HeartPy_interpolated'].mean())/data['HR_HeartPy_interpolated'].std()
    
    """OLD
    standardizedData_SC = (data['Skin_Conductance'] - data['Skin_Conductance'].mean())/data['Skin_Conductance'].std()
    location = data.columns.get_loc("Skin_Conductance")
    data.insert(loc=location, column='Skin_Conductance_STD', value = standardizedData_SC)
    
    standardizedData_SR = (data['Skin_Resistance'] - data['Skin_Resistance'].mean())/data['Skin_Resistance'].std()
    location = data.columns.get_loc("Skin_Resistance")
    data.insert(loc=location, column='Skin_Resistance_STD', value = standardizedData_SR)
    
    standardizedData_CONS_HR = (data['HR_CONSENSYS_interpolated'] - data['HR_CONSENSYS_interpolated'].mean())/data['HR_CONSENSYS_interpolated'].std()
    location = data.columns.get_loc("HR_CONSENSYS_interpolated")
    data.insert(loc=location, column='HR_CONSENSYS_STD', value = standardizedData_CONS_HR)
    
    standardizedData_HP_HR = (data['HR_HeartPy_interpolated'] - data['HR_HeartPy_interpolated'].mean())/data['HR_HeartPy_interpolated'].std()
    location = data.columns.get_loc("HR_HeartPy_interpolated")
    data.insert(loc=location, column='HR_HP_STD', value = standardizedData_HP_HR)
    """
    return data

### 2.6 : ***normalize data (0-1)***

In [18]:
def normalizeData(data):
    
    data['Skin_Conductance_NORM'] = (data['Skin_Conductance'] - data['Skin_Conductance'].min())/(data['Skin_Conductance'].max()-data['Skin_Conductance'].min())
    data['Skin_Resistance_NORM'] = (data['Skin_Resistance'] - data['Skin_Resistance'].min())/(data['Skin_Resistance'].max()-data['Skin_Resistance'].min())
    data['HR_CONSENSYS_NORM'] = (data['HR_CONSENSYS_interpolated'] - data['HR_CONSENSYS_interpolated'].min())/(data['HR_CONSENSYS_interpolated'].max()-data['HR_CONSENSYS_interpolated'].min())
    data['HR_HP_NORM'] = (data['HR_HeartPy_interpolated'] - data['HR_HeartPy_interpolated'].min())/(data['HR_HeartPy_interpolated'].max()-data['HR_HeartPy_interpolated'].min())
    
    """OLD
    normalizedData_SC = (data['Skin_Conductance'] - data['Skin_Conductance'].min())/(data['Skin_Conductance'].max()-data['Skin_Conductance'].min())    
    location = data.columns.get_loc("Skin_Conductance_STD")
    data.insert(loc=location, column='Skin_Conductance_NORM', value = normalizedData_SC)
    
    normalizedData_SR = (data['Skin_Resistance'] - data['Skin_Resistance'].min())/(data['Skin_Resistance'].max()-data['Skin_Resistance'].min())
    location = data.columns.get_loc("Skin_Resistance_STD")
    data.insert(loc=location, column='Skin_Resistance_NORM', value = normalizedData_SR)
    
    normalizedData_CONS_HR = (data['HR_CONSENSYS_interpolated'] - data['HR_CONSENSYS_interpolated'].min())/(data['HR_CONSENSYS_interpolated'].max()-data['HR_CONSENSYS_interpolated'].min())
    location = data.columns.get_loc("HR_CONSENSYS_STD")
    data.insert(loc=location, column='HR_CONSENSYS_NORM', value = normalizedData_CONS_HR)
    
    normalizedData_HP_HR = (data['HR_HeartPy_interpolated'] - data['HR_HeartPy_interpolated'].min())/(data['HR_HeartPy_interpolated'].max()-data['HR_HeartPy_interpolated'].min())
    location = data.columns.get_loc("HR_HP_STD")
    data.insert(loc=location, column='HR_HP_NORM', value = normalizedData_HP_HR)
    """    
    return data

### 2.7 process EDA using cvxEDA

I'm not sure why I set solver options to these features... no recollection of it

In [19]:
def processEDA(data, testMode = False):
    from cvxopt import solvers
    solvers.options['show_progress'] = True
    solvers.options['abstol'] = 1e-02
    solvers.options['feastol'] = 1e-02
    solvers.options['reltol']=1e-02
    # had to comment out line 107 in cvxEDA.py : cv.solvers.options.clear() to enable seting options on cvxopt 
    
    freq = 1.0 / 128.0
    
    """
    cvxEDA(y, delta, tau0=2., tau1=0.7, delta_knot=10., alpha=8e-4, gamma=1e-2,
           solver=None, options={'reltol':1e-9}):
    CVXEDA Convex optimization approach to electrodermal activity processing
    This function implements the cvxEDA algorithm described in "cvxEDA: a
    Convex Optimization Approach to Electrodermal Activity Processing"
    (http://dx.doi.org/10.1109/TBME.2015.2474131, also available from the
    authors' homepages).
    Arguments:
       y: observed EDA signal (we recommend normalizing it: y = zscore(y))
       delta: sampling interval (in seconds) of y
       tau0: slow time constant of the Bateman function
       tau1: fast time constant of the Bateman function
       delta_knot: time between knots of the tonic spline function
       alpha: penalization for the sparse SMNA driver
       gamma: penalization for the tonic spline coefficients
       solver: sparse QP solver to be used, see cvxopt.solvers.qp
       options: solver options, see:
                http://cvxopt.org/userguide/coneprog.html#algorithm-parameters
    """
    
    [phasicComponentData, p, tonicComponentData, l, d, e, obj] = cvxEDA(data['Skin_Conductance_STD'], freq, options = solvers.options) 
    # new way: , options = solvers.options
    """
       r: phasic component
       p: sparse SMNA driver of phasic component
       t: tonic component
       l: coefficients of tonic spline
       d: offset and slope of the linear drift term
       e: model residuals
       obj: value of objective function being minimized (eq 15 of paper)
    """
    if testMode:
        print("TEST1 processEDA ===============")
        print(data.head(2))
        print("TEST1 processEDA END============")
    
    location = data.columns.get_loc("Skin_Conductance_STD")
    
    #data.insert(loc=location, column='Skin_Conductance_Tonic_CVX', value = tonicComponentData)
    
    #print("TEST2 processEDA ===============")
    #print(data.head(2))
    #print("TEST2 processEDA END============")
    if testMode:
        print("TEST3 processEDA =============== concat test")
    
    #print(type(phasicComponentData))
    #phasicComponentData = pd.DataFrame(phasicComponentData)
    #print(type(phasicComponentData))
    #pd.concat([data, phasicComponentData]) # .insert(loc=location, column='Skin_Conductance_Phasic_CVX', value = phasicComponentData)
    #cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid
    
    data['Skin_Conductance_Phasic_CVX'] = phasicComponentData[:]
    data['Skin_Conductance_Tonic_CVX'] = tonicComponentData[:]
    
    if testMode:
        print(data.head(2))
        print("TEST3 processEDA END============")
    
    #location = data.columns.get_loc("Skin_Conductance_STD")
    #data.insert(loc=location, column='Skin_Conductance_Phasic_CVX', value = phasicComponentData)

    
    return data

### 2.8 smooth EDA and HR using centered SMA

In [20]:
def smoothHRandSC(data):
    window1 = 128 # 1s
    window2 = 3*128 # 3s
    window3 = 60*128 # 1m
    windows = [window1,window2,window3]
    
    listOfParametersToSmooth = ['Skin_Conductance',
                                "Skin_Conductance_NORM",
                                'Skin_Conductance_STD', 
                                "Skin_Conductance_Phasic_CVX",
                                'Skin_Resistance_NORM',
                                'Skin_Resistance_STD', 
                                'Skin_Resistance',
                                'PPG_to_HR_CONSENSYS', 
                                'HR_CONSENSYS_NORM', 
                                'HR_CONSENSYS_STD', 
                                'HR_CONSENSYS_interpolated',
                                'bpm',
                                'HR_HeartPy_interpolated',
                                'HR_HP_NORM',
                                'HR_HP_STD']
    
    for parameter in listOfParametersToSmooth:
        for window in windows:
            sma = pd.Series(data[parameter]).rolling(window = window, center = True).mean()
            columnName = parameter + "_SMA(" + str(window/128) + "s)"
            data[columnName] = sma
        
    return data

In [21]:
def smoothHRandSCewma(data, testMode = False):
    
    
    """
    added NewDF due to:
    <ipython-input-20-666b3cc13adb>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[columnName] = data[parameter].ewm(alpha = alpha).mean()
    """
    NewDF = data.copy()
    
    alpha1 = 0.0001 # alpha proportional to the lenght of TS
    alpha2 = 0.001 # mid range
    alpha3 = 0.01  # near sample
    alphas = [alpha1,alpha2,alpha3]
    listOfParametersToSmooth = ['Skin_Conductance',
                                "Skin_Conductance_NORM",
                                'Skin_Conductance_STD', 
                                "Skin_Conductance_Phasic_CVX",
                                'Skin_Resistance_NORM',
                                'Skin_Resistance_STD', 
                                'Skin_Resistance',
                                'PPG_to_HR_CONSENSYS', 
                                'HR_CONSENSYS_NORM', 
                                'HR_CONSENSYS_STD', 
                                'HR_CONSENSYS_interpolated',
                                'bpm',
                                'HR_HeartPy_interpolated',
                                'HR_HP_NORM',
                                'HR_HP_STD']
    
    for parameter in listOfParametersToSmooth:
        for alpha in alphas:
            columnName = "{}_EWMA({})".format(parameter , alpha)
            if testMode:
                print(columnName)
            NewDF[columnName] = data[parameter].ewm(alpha = alpha).mean()
        
    return NewDF

## 2.8 Add 1st order differentiation results column

In [22]:
def diff1st(data):
    # could use a validated method from np or pd here np.diff:
    
    parametersToDifferentiate = ["HR_CONSENSYS_interpolated","HR_CONSENSYS_STD","HR_CONSENSYS_NORM",
                                 "HR_HeartPy_interpolated","HR_HP_STD","HR_HP_NORM",
                                 "Skin_Conductance","Skin_Conductance_STD","Skin_Conductance_NORM",
                                 "Skin_Resistance","Skin_Resistance_STD","Skin_Resistance_NORM"]
    
    for parameter in parametersToDifferentiate:
        """
        print("TEST ============== .diff()")
        print(parameter) 
        print(type(data[parameter]))
        print(data[parameter][1])
        print(type(data[parameter][1]))
        """
        
        try:
            diffData = data[parameter].diff() # 1st order
        except Exception as e: 
            print("error_3__1 : diff()")
            print(e) 
            print(parameter) 
        """
        print(diffData.head(2))
        print("TEST END ============== .diff()")
        """
        columnName = parameter + "_DIFF(1)"
        data[columnName] = diffData
        
    """
    dataLength = len(data["HR_CONSENSYS_interpolated"])
    HR_CONSENSYS_DIFF1 = np.zeros(dataLength)
    HR_HP_DIFF1 = np.zeros(dataLength)
    SC_DIFF1 = np.zeros(dataLength)
    SR_DIFF1 = np.zeros(dataLength)
        
    for i in range(0,dataLength):
        if(i==0):
            HR_CONSENSYS_DIFF1[i] = 0
            HR_HP_DIFF1[i] = 0
            SC_DIFF1[i] = 0
            SR_DIFF1[i] = 0
        else:
            HR_CONSENSYS_DIFF1[i] = data["HR_CONSENSYS_interpolated"][i] - data["HR_CONSENSYS_interpolated"][i-1]
            HR_HP_DIFF1[i] = data["HR_HeartPy_interpolated"][i] - data["HR_HeartPy_interpolated"][i-1]
            SC_DIFF1[i] = data["Skin_Conductance"][i] - data["Skin_Conductance"][i-1]
            SR_DIFF1[i] = data["Skin_Resistance"][i] - data["Skin_Resistance"][i-1]  
    
    location = data.columns.get_loc("HR_CONSENSYS_interpolated") + 2
    data.insert(loc=location, column='HR_CONSENSYS_DIFF1', value = HR_CONSENSYS_DIFF1)
    location = data.columns.get_loc("HR_HeartPy_interpolated") + 2
    data.insert(loc=location, column='HR_HP_DIFF1', value = HR_HP_DIFF1)
    location = data.columns.get_loc("Skin_Conductance") + 2
    data.insert(loc=location, column='Skin_Conductance_DIFF1', value = SC_DIFF1)
    location = data.columns.get_loc("Skin_Resistance") + 2
    data.insert(loc=location, column='Skin_Resistance_DIFF1', value = SR_DIFF1)
    """    
        
    return data

## 3 : Merge Participant and Environment data:

### 3.1 : merge_asof on nearest timestamp (tolerance = 10ms)

In [23]:
def mergeAll(data1, data2):
    timeS = data1['timestamp'][1] - data2['timestamp'][1]
    
    all_data = pd.merge_asof(data1, data2, on="timestamp", direction='nearest', tolerance = pd.Timedelta('50ms'))
    #10ms produces NA as there is not enough overlap to account for lags in the environment data which goes to 40ms at times

    return all_data

### 3.2 : Add absolute experiment time:

function deprecated as the deltatime object was converted to a float in the spreadsheet (i.e.: 0.0000000810185185185185) 
I could format it as a string or dedicate more time for converting it to a timestamp but it was only an additional feature
So I have abandoned it for now.

In [24]:
def absoluteTime(data):
    startTime = data["timestamp"].iloc[0]
    #print(data["timestamp"].head())
    #print(type(data["timestamp"]))
    #print(data['timestamp'] - startTime)
    #print(type(data['timestamp'] - startTime))
    
    try:
        #data["TEMP_Time"] = pd.to_datetime(data['timestamp'] - startTime) ###### rewrote to retain datetime data
        #... delta time is not time stamp
        # if I really really need it:
        # data data["TEMP_Time"] = data['timestamp'] + data["TEMP_Time"]
        #???
        data["TEMP_Time"] = data['timestamp'].iloc[:] - startTime
    except Exception as e: 
        print("error_3__1 : converting time")
        print(e)
        
    print("TEST1===============================================")
    print(data["TEMP_Time"].iloc[-1])
    print(type(data["TEMP_Time"].iloc[-1]))
    print("TEST1===END=========================================")
    
    try:
        data["Experiment_duration"] = data["TEMP_Time"]
        #data["Experiment_duration"] = data["TEMP_Time"].dt.strftime('%M:%S.%f')
        #error 'TimedeltaProperties' object has no attribute 'strftime'
        #data["Experiment_duration"] = pd.to_datetime(data['timestamp'] - startTime).dt.strftime('%M:%S.%f')
        # problem is that I want it nicely formated but its a delta time object
        """
        I could write a function to process that with microsseconds similar to divmod below:
        s = 13420
        hours, remainder = divmod(s, 3600)
        minutes, seconds = divmod(remainder, 60)
        print '{:02}:{:02}:{:02}'.format(int(hours), int(minutes), int(seconds))
        # result: 03:43:40"""
    except Exception as e: 
        print("error_3__2 : converting time")
        print(e)        
    return data

## 4 : Pull demographic and questionaire data

***!!! This code runs to early should be the last one to be merged if merged at all? maybe use a second DF and store as a second sheet??***

In [25]:
def pullDemoQdata(ID,GR):
    filePath = 'RawData/Qualitative data and participant demographics/' + 'id_' + str(ID) + '-Gr_' + str(GR) + '.xlsx'
    dataPath = os.path.join(os.getcwd(), filePath)
    #EXPERIMENT_1_DATA\MSSQandIPQ
    data = pd.read_excel(dataPath, sheet_name='Basic', header = None)
    allDemoData = {
        'ID' : data.iloc[0,1],
        'Group' : data.iloc[1,1],
        'Gender' : data.iloc[2,1],
        'Age' : data.iloc[3,1],
        'MSSQ' : data.iloc[5,1],
        'IPQ-Avrg' : data.iloc[8,1],
        'IPQ-G1' : data.iloc[9,1],
        'IPQ-SP' : data.iloc[10,1],
        'IPQ-INV' : data.iloc[11,1],
        'IPQ-REAL' : data.iloc[12,1]
    }
    
    demoData = pd.DataFrame.from_dict(allDemoData, orient='index')
    
    return demoData

## 5 : Export

In [26]:
def saveAllData(all_data, ID, GR, demographicData, save = True):
    save = saveGlobal
    # print(all_data.columns)
    
    data_to_save = pd.DataFrame()

    #data_to_save["Raw_Time[HH:MM:ss.us]"] = all_data["Experiment_duration"]
    # LOOKS LIKE .copy() didn't fixed the problem
    data_to_save["Frame_count"] = all_data["Frame_count"].copy()

    data_to_save["Phase"] = all_data["Phase"].copy()
    data_to_save["Time_in_Phase[s]"] = all_data["Time_in_Phase"].copy()
    data_to_save["Real_Time_in_Phase[s]"] = all_data["Real_Time_in_Phase"].copy()
    data_to_save["Trial_Number[int]"] = all_data["Trial_Number"].copy()
    data_to_save["Trial_Time[s]"] = all_data["Trial_Time"].copy()
    data_to_save["Period_Type"] = all_data["Period_Type"].copy()
    data_to_save["Time_in_Period[s]"] = all_data["Time_in_Period"].copy()
    data_to_save["Fidelity"] = all_data["Fidelity"].copy()
    data_to_save["Model"] = all_data["Model"].copy()
    data_to_save["Saturation"] = all_data["Saturation"].copy()
    data_to_save["FOV[deg]"] = all_data["FOV"].copy()
    data_to_save["Resolution[px]"] = all_data["Resolution"].copy()
    data_to_save["Pixel_count[px]"] = all_data["Pixel_count"].copy()
    data_to_save["Thread_sleep[ms]"] = all_data["Thread_sleep"].copy()
    data_to_save["FPS"] = all_data["FPS_real"].copy()
    data_to_save["Audio_clip"] = all_data["Audio_clip"].copy()
    '''    
    data_to_save["Audio_clip"] = all_data["Audio_clip"]
    audio clip data droped as it didn't contain any interesting information
    audio clip Audio_1 was played during the break phase which is removed from the analysed dataset 
    audio clip Audio_2 was played at the end of the experiment after data capture was finished
    ...
    WAS ADDED BACK IN!!!!!!!!!!!
    '''

    data_to_save["Skin_Conductance[uS]"] = all_data["Skin_Conductance"].copy()
    data_to_save["Skin_Conductance_DIFF(1)"] = all_data["Skin_Conductance_DIFF(1)"].copy()
    
    data_to_save['Skin_Conductance_SMA(1s)'] = all_data['Skin_Conductance_SMA(1.0s)'].copy()
    data_to_save['Skin_Conductance_SMA(3s)'] = all_data['Skin_Conductance_SMA(3.0s)'].copy()
    data_to_save['Skin_Conductance_SMA(60s)'] = all_data['Skin_Conductance_SMA(60.0s)'].copy()
    data_to_save['Skin_Conductance_EWMA(0.01)'] = all_data['Skin_Conductance_EWMA(0.01)'].copy()
    data_to_save['Skin_Conductance_EWMA(0.001)'] = all_data['Skin_Conductance_EWMA(0.001)'].copy()
    data_to_save['Skin_Conductance_EWMA(0.0001)'] = all_data['Skin_Conductance_EWMA(0.0001)'].copy()

    data_to_save["Skin_Conductance_STD"] = all_data["Skin_Conductance_STD"].copy()
    data_to_save["Skin_Conductance_STD_DIFF(1)"] = all_data["Skin_Conductance_STD_DIFF(1)"].copy()
    
    data_to_save['Skin_Conductance_STD_SMA(1s)'] = all_data['Skin_Conductance_STD_SMA(1.0s)'].copy()
    data_to_save['Skin_Conductance_STD_SMA(3s)'] = all_data['Skin_Conductance_STD_SMA(3.0s)'].copy()
    data_to_save['Skin_Conductance_STD_SMA(60s)'] = all_data['Skin_Conductance_STD_SMA(60.0s)'].copy()
    data_to_save['Skin_Conductance_STD_EWMA(0.01)'] = all_data['Skin_Conductance_STD_EWMA(0.01)'].copy()
    data_to_save['Skin_Conductance_STD_EWMA(0.001)'] = all_data['Skin_Conductance_STD_EWMA(0.001)'].copy()
    data_to_save['Skin_Conductance_STD_EWMA(0.0001)'] = all_data['Skin_Conductance_STD_EWMA(0.0001)'].copy()
    
    data_to_save["Skin_Conductance_NORM"] = all_data["Skin_Conductance_NORM"].copy()
    data_to_save["Skin_Conductance_NORM_DIFF(1)"] = all_data["Skin_Conductance_NORM_DIFF(1)"].copy()
    
    data_to_save["Skin_Conductance_NORM_SMA(1s)"] = all_data["Skin_Conductance_NORM_SMA(1.0s)"].copy()
    data_to_save["Skin_Conductance_NORM_SMA(3s)"] = all_data["Skin_Conductance_NORM_SMA(3.0s)"].copy()
    data_to_save['Skin_Conductance_NORM_SMA(60s)'] = all_data['Skin_Conductance_NORM_SMA(60.0s)'].copy()
    data_to_save['Skin_Conductance_NORM_EWMA(0.01)'] = all_data['Skin_Conductance_NORM_EWMA(0.01)'].copy()
    data_to_save['Skin_Conductance_NORM_EWMA(0.001)'] = all_data['Skin_Conductance_NORM_EWMA(0.001)'].copy()
    data_to_save['Skin_Conductance_NORM_EWMA(0.0001)'] = all_data['Skin_Conductance_NORM_EWMA(0.0001)'].copy()
    
    data_to_save["Skin_Resistance[kOhms]"] = all_data["Skin_Resistance"].copy()
    data_to_save["Skin_Resistance_DIFF(1)"] = all_data["Skin_Resistance_DIFF(1)"].copy()
    
    data_to_save["Skin_Resistance_SMA(1s)"] = all_data["Skin_Resistance_SMA(1.0s)"].copy()
    data_to_save["Skin_Resistance_SMA(3s)"] = all_data["Skin_Resistance_SMA(3.0s)"].copy()
    data_to_save["Skin_Resistance_SMA(60s)"] = all_data["Skin_Resistance_SMA(60.0s)"].copy()
    data_to_save['Skin_Resistance_EWMA(0.01)'] = all_data['Skin_Resistance_EWMA(0.01)'].copy()
    data_to_save['Skin_Resistance_EWMA(0.001)'] = all_data['Skin_Resistance_EWMA(0.001)'].copy()
    data_to_save['Skin_Resistance_EWMA(0.0001)'] = all_data['Skin_Resistance_EWMA(0.0001)'].copy()
    
    data_to_save["Skin_Resistance_STD"] = all_data["Skin_Resistance_STD"].copy()
    data_to_save["Skin_Resistance_STD_DIFF(1)"] = all_data["Skin_Resistance_STD_DIFF(1)"].copy()
    
    data_to_save["Skin_Resistance_STD_SMA(1s)"] = all_data["Skin_Resistance_STD_SMA(1.0s)"].copy()   
    data_to_save["Skin_Resistance_STD_SMA(3s)"] = all_data["Skin_Resistance_STD_SMA(3.0s)"].copy()  
    data_to_save["Skin_Resistance_STD_SMA(60s)"] = all_data["Skin_Resistance_STD_SMA(60.0s)"].copy()
    data_to_save['Skin_Resistance_STD_EWMA(0.01)'] = all_data['Skin_Resistance_STD_EWMA(0.01)'].copy()
    data_to_save['Skin_Resistance_STD_EWMA(0.001)'] = all_data['Skin_Resistance_STD_EWMA(0.001)'].copy()
    data_to_save['Skin_Resistance_STD_EWMA(0.0001)'] = all_data['Skin_Resistance_STD_EWMA(0.0001)'].copy()
    
    data_to_save["Skin_Resistance_NORM"] = all_data["Skin_Resistance_NORM"].copy()
    data_to_save["Skin_Resistance_NORM_DIFF(1)"] = all_data["Skin_Resistance_NORM_DIFF(1)"].copy()
    
    data_to_save["Skin_Resistance_NORM_SMA(1s)"] = all_data["Skin_Resistance_NORM_SMA(1.0s)"].copy()
    data_to_save["Skin_Resistance_NORM_SMA(3s)"] = all_data["Skin_Resistance_NORM_SMA(3.0s)"].copy()
    data_to_save["Skin_Resistance_NORM_SMA(60s)"] = all_data["Skin_Resistance_NORM_SMA(60.0s)"].copy()
    data_to_save['Skin_Resistance_NORM_EWMA(0.01)'] = all_data['Skin_Resistance_NORM_EWMA(0.01)'].copy()
    data_to_save['Skin_Resistance_NORM_EWMA(0.001)'] = all_data['Skin_Resistance_NORM_EWMA(0.001)'].copy()
    data_to_save['Skin_Resistance_NORM_EWMA(0.0001)'] = all_data['Skin_Resistance_NORM_EWMA(0.0001)'].copy()
    
    data_to_save["Skin_Conductance_Phasic_CVX"] = all_data["Skin_Conductance_Phasic_CVX"].copy()
    data_to_save["Skin_Conductance_Phasic_CVX_SMA(1s)"] = all_data["Skin_Conductance_Phasic_CVX_SMA(1.0s)"].copy()
    data_to_save["Skin_Conductance_Phasic_CVX_SMA(3s)"] = all_data["Skin_Conductance_Phasic_CVX_SMA(3.0s)"].copy()
    data_to_save["Skin_Conductance_Phasic_CVX_SMA(60s)"] = all_data["Skin_Conductance_Phasic_CVX_SMA(60.0s)"].copy()
    data_to_save['Skin_Conductance_Phasic_CVX_EWMA(0.01)'] = all_data['Skin_Conductance_Phasic_CVX_EWMA(0.01)'].copy()
    data_to_save['Skin_Conductance_Phasic_CVX_EWMA(0.001)'] = all_data['Skin_Conductance_Phasic_CVX_EWMA(0.001)'].copy()
    data_to_save['Skin_Conductance_Phasic_CVX_EWMA(0.0001)'] = all_data['Skin_Conductance_Phasic_CVX_EWMA(0.0001)'].copy()
    
    data_to_save["Skin_Conductance_Tonic_CVX"] = all_data["Skin_Conductance_Tonic_CVX"].copy()
    
    data_to_save["HR_CONSENSYS[BPM]"] = all_data["PPG_to_HR_CONSENSYS"].copy()
    
    data_to_save["HR_CONSENSYS_SMA(1s)"] = all_data["PPG_to_HR_CONSENSYS_SMA(1.0s)"].copy()
    data_to_save["HR_CONSENSYS_SMA(3s)"] = all_data["PPG_to_HR_CONSENSYS_SMA(3.0s)"].copy()
    data_to_save["HR_CONSENSYS_SMA(60s)"] = all_data["PPG_to_HR_CONSENSYS_SMA(60.0s)"].copy()
    data_to_save['HR_CONSENSYS_EWMA(0.01)'] = all_data['PPG_to_HR_CONSENSYS_EWMA(0.01)'].copy()
    data_to_save['HR_CONSENSYS_EWMA(0.001)'] = all_data['PPG_to_HR_CONSENSYS_EWMA(0.001)'].copy()
    data_to_save['HR_CONSENSYS_EWMA(0.0001)'] = all_data['PPG_to_HR_CONSENSYS_EWMA(0.0001)'].copy()
    
    
    data_to_save["HR_CONSENSYS_interpolated[BPM]"] = all_data["HR_CONSENSYS_interpolated"].copy()
    data_to_save["HR_CONSENSYS_interpolated_DIFF(1)"] = all_data["HR_CONSENSYS_interpolated_DIFF(1)"].copy()
    
    data_to_save["HR_CONSENSYS_interpolated_SMA(1s)"] = all_data["HR_CONSENSYS_interpolated_SMA(1.0s)"].copy()
    data_to_save["HR_CONSENSYS_interpolated_SMA(3s)"] = all_data["HR_CONSENSYS_interpolated_SMA(3.0s)"].copy()
    data_to_save["HR_CONSENSYS_interpolated_SMA(60s)"] = all_data["HR_CONSENSYS_interpolated_SMA(60.0s)"].copy()
    data_to_save['HR_CONSENSYS_interpolated_EWMA(0.01)'] = all_data['HR_CONSENSYS_interpolated_EWMA(0.01)'].copy()
    data_to_save['HR_CONSENSYS_interpolated_EWMA(0.001)'] = all_data['HR_CONSENSYS_interpolated_EWMA(0.001)'].copy()
    data_to_save['HR_CONSENSYS_interpolated_EWMA(0.0001)'] = all_data['HR_CONSENSYS_interpolated_EWMA(0.0001)'].copy()
    
    data_to_save["HR_CONSENSYS_interpolated_STD"] = all_data["HR_CONSENSYS_STD"].copy()
    data_to_save["HR_CONSENSYS_interpolated_STD_DIFF(1)"] = all_data["HR_CONSENSYS_STD_DIFF(1)"].copy()
    
    data_to_save["HR_CONSENSYS_interpolated_STD_SMA(1s)"] = all_data["HR_CONSENSYS_STD_SMA(1.0s)"].copy()
    data_to_save["HR_CONSENSYS_interpolated_STD_SMA(3s)"] = all_data["HR_CONSENSYS_STD_SMA(3.0s)"].copy()
    data_to_save["HR_CONSENSYS_interpolated_STD_SMA(60s)"] = all_data["HR_CONSENSYS_STD_SMA(60.0s)"].copy()
    data_to_save['HR_CONSENSYS_interpolated_STD_EWMA(0.01)'] = all_data['HR_CONSENSYS_STD_EWMA(0.01)'].copy()
    data_to_save['HR_CONSENSYS_interpolated_STD_EWMA(0.001)'] = all_data['HR_CONSENSYS_STD_EWMA(0.001)'].copy()
    data_to_save['HR_CONSENSYS_interpolated_STD_EWMA(0.0001)'] = all_data['HR_CONSENSYS_STD_EWMA(0.0001)'].copy()
    
    data_to_save["HR_CONSENSYS_interpolated_NORM"] = all_data["HR_CONSENSYS_NORM"].copy()
    data_to_save["HR_CONSENSYS_interpolated_NORM_DIFF(1)"] = all_data["HR_CONSENSYS_NORM_DIFF(1)"].copy()
    
    data_to_save["HR_CONSENSYS_interpolated_NORM_SMA(1s)"] = all_data["HR_CONSENSYS_NORM_SMA(1.0s)"].copy()
    data_to_save["HR_CONSENSYS_interpolated_NORM_SMA(3s)"] = all_data["HR_CONSENSYS_NORM_SMA(3.0s)"].copy()
    data_to_save["HR_CONSENSYS_interpolated_NORM_SMA(60s)"] = all_data["HR_CONSENSYS_NORM_SMA(60.0s)"].copy() 
    data_to_save['HR_CONSENSYS_interpolated_NORM_EWMA(0.01)'] = all_data['HR_CONSENSYS_NORM_EWMA(0.01)'].copy()
    data_to_save['HR_CONSENSYS_interpolated_NORM_EWMA(0.001)'] = all_data['HR_CONSENSYS_NORM_EWMA(0.001)'].copy()
    data_to_save['HR_CONSENSYS_interpolated_NORM_EWMA(0.0001)'] = all_data['HR_CONSENSYS_NORM_EWMA(0.0001)'].copy()
    
    data_to_save["IBI_CONSENSYS[ms]"] = all_data['PPG_IBI'].copy()
    data_to_save["IBI_CONSENSYS_interpolated[ms]"] = all_data['IBI_CONSENSYS_interpolated'].copy()

    data_to_save["HR_HeartPy[BPM]"] = all_data["bpm"].copy()
    data_to_save["HR_HeartPy_SMA(1s)"] = all_data['bpm_SMA(1.0s)'].copy()
    data_to_save["HR_HeartPy_SMA(3s)"] = all_data["bpm_SMA(3.0s)"].copy()
    data_to_save["HR_HeartPy_SMA(60s)"] = all_data['bpm_SMA(60.0s)'].copy()
    data_to_save['HR_HeartPy_EWMA(0.01)'] = all_data['bpm_EWMA(0.01)'].copy()
    data_to_save['HR_HeartPy_EWMA(0.001)'] = all_data['bpm_EWMA(0.001)'].copy()
    data_to_save['HR_HeartPy_EWMA(0.0001)'] = all_data['bpm_EWMA(0.0001)'].copy()
    
    data_to_save["HR_HeartPy_interpolated[BPM]"] = all_data["HR_HeartPy_interpolated"].copy()
    data_to_save["HR_HeartPy_interpolated_DIFF(1)"] = all_data["HR_HeartPy_interpolated_DIFF(1)"].copy()
    
    data_to_save["HR_HeartPy_interpolated_SMA(1s)"] = all_data["HR_HeartPy_interpolated_SMA(1.0s)"].copy()
    data_to_save["HR_HeartPy_interpolated_SMA(3s)"] = all_data["HR_HeartPy_interpolated_SMA(3.0s)"].copy()
    data_to_save["HR_HeartPy_interpolated_SMA(60s)"] = all_data['HR_HeartPy_interpolated_SMA(60.0s)'].copy()
    data_to_save['HR_HeartPy_interpolated_EWMA(0.01)'] = all_data['HR_HeartPy_interpolated_EWMA(0.01)'].copy()
    data_to_save['HR_HeartPy_interpolated_EWMA(0.001)'] = all_data['HR_HeartPy_interpolated_EWMA(0.001)'].copy()
    data_to_save['HR_HeartPy_interpolated_EWMA(0.0001)'] = all_data['HR_HeartPy_interpolated_EWMA(0.0001)'].copy()
    
    data_to_save["HR_HeartPy_interpolated_STD"] = all_data["HR_HP_STD"].copy()
    data_to_save["HR_HeartPy_interpolated_STD_DIFF(1)"] = all_data["HR_HP_STD_DIFF(1)"].copy()
    
    data_to_save["HR_HeartPy_interpolated_STD_SMA(1s)"] = all_data["HR_HP_STD_SMA(1.0s)"].copy()
    data_to_save["HR_HeartPy_interpolated_STD_SMA(3s)"] = all_data["HR_HP_STD_SMA(3.0s)"].copy()
    data_to_save["HR_HeartPy_interpolated_STD_SMA(60s)"] = all_data['HR_HP_STD_SMA(60.0s)'].copy()
    data_to_save['HR_HeartPy_interpolated_STD_EWMA(0.01)'] = all_data['HR_HP_STD_EWMA(0.01)'].copy()
    data_to_save['HR_HeartPy_interpolated_STD_EWMA(0.001)'] = all_data['HR_HP_STD_EWMA(0.001)'].copy()
    data_to_save['HR_HeartPy_interpolated_STD_EWMA(0.0001)'] = all_data['HR_HP_STD_EWMA(0.0001)'].copy()
    
    data_to_save["HR_HeartPy_interpolated_NORM"] = all_data["HR_HP_NORM"].copy()
    data_to_save["HR_HeartPy_interpolated_NORM_DIFF(1)"] = all_data["HR_HP_NORM_DIFF(1)"].copy()
    
    data_to_save["HR_HeartPy_interpolated_NORM_SMA(1s)"] = all_data["HR_HP_NORM_SMA(1.0s)"].copy()
    data_to_save["HR_HeartPy_interpolated_NORM_SMA(3s)"] = all_data["HR_HP_NORM_SMA(3.0s)"].copy()
    data_to_save["HR_HeartPy_interpolated_NORM_SMA(60s)"] = all_data['HR_HP_NORM_SMA(60.0s)'].copy()
    data_to_save['HR_HeartPy_interpolated_NORM_EWMA(0.01)'] = all_data['HR_HP_NORM_EWMA(0.01)'].copy()
    data_to_save['HR_HeartPy_interpolated_NORM_EWMA(0.001)'] = all_data['HR_HP_NORM_EWMA(0.001)'].copy()
    data_to_save['HR_HeartPy_interpolated_NORM_EWMA(0.0001)'] = all_data['HR_HP_NORM_EWMA(0.0001)'].copy()  
    
    data_to_save["IBI_HeartPy[ms]"] = all_data["ibi"].copy()
    data_to_save["IBI_HeartPy_interpolated[ms]"] = all_data["IBI_HeartPy_interpolated"].copy()
    
    data_to_save["SDNN_HeartPy[ms]"] = all_data["sdnn"].copy()
    data_to_save["SDNN_HeartPy_interpolated[ms]"] = all_data["SDNN_HeartPy_interpolated"].copy()
    
    data_to_save["SDSD_HeartPy[ms]"] = all_data["sdsd"].copy()
    data_to_save["SDSD_HeartPy_interpolated[ms]"] = all_data["SDSD_HeartPy_interpolated"].copy()
    
    data_to_save["RMSSD_HeartPy[ms]"] = all_data["rmssd"]
    data_to_save["RMSSD_HeartPy_interpolated[ms]"] = all_data["RMSSD_HeartPy_interpolated"]
    
    data_to_save["pNN20_HeartPy[%]"] = all_data["pnn20"].copy()
    data_to_save["pNN20_HeartPy_interpolated[%]"] = all_data["pNN20_HeartPy_interpolated"].copy()
    
    data_to_save["pNN50_HeartPy[%]"] = all_data["pnn50"].copy()
    data_to_save["pNN50_HeartPy_interpolated[%]"] = all_data["pNN50_HeartPy_interpolated"].copy()
    
    data_to_save["Pressure[kPa]"] = all_data["Pressure"].copy()
    data_to_save["Temperature[C]"] = all_data["Temperature"].copy()
    
    """OLD
    #data_to_save["Raw_Time[HH:MM:ss.us]"] = all_data["Experiment_duration"]
    data_to_save["Frame_count"] = all_data["Frame_count"]

    data_to_save["Phase"] = all_data["Phase"] 
    data_to_save["Time_in_Phase[s]"] = all_data["Time_in_Phase"]
    data_to_save["Real_Time_in_Phase[s]"] = all_data["Real_Time_in_Phase"]
    data_to_save["Trial_Number[int]"] = all_data["Trial_Number"]
    data_to_save["Trial_Time[s]"] = all_data["Trial_Time"]
    data_to_save["Period_Type"] = all_data["Period_Type"]
    data_to_save["Time_in_Period[s]"] = all_data["Time_in_Period"]
    data_to_save["Fidelity"] = all_data["Fidelity"]
    data_to_save["Model"] = all_data["Model"]
    data_to_save["Saturation"] = all_data["Saturation"]
    data_to_save["FOV[deg]"] = all_data["FOV"]
    data_to_save["Resolution[px]"] = all_data["Resolution"]
    data_to_save["Pixel_count[px]"] = all_data["Pixel_count"]
    data_to_save["Thread_sleep[ms]"] = all_data["Thread_sleep"]
    data_to_save["FPS"] = all_data["FPS_real"]
    data_to_save["Audio_clip"] = all_data["Audio_clip"]
    '''    
    data_to_save["Audio_clip"] = all_data["Audio_clip"]
    audio clip data droped as it didn't contain any interesting information
    audio clip Audio_1 was played during the break phase which is removed from the analysed dataset 
    audio clip Audio_2 was played at the end of the experiment after data capture was finished
    WAS ADDED BACK IN!!!!!!!!!!!
    '''

    data_to_save["Skin_Conductance[uS]"] = all_data["Skin_Conductance"]
    data_to_save["Skin_Conductance_DIFF(1)"] = all_data["Skin_Conductance_DIFF(1)"]
    
    data_to_save['Skin_Conductance_SMA(1s)'] = all_data['Skin_Conductance_SMA(1.0s)']
    data_to_save['Skin_Conductance_SMA(3s)'] = all_data['Skin_Conductance_SMA(3.0s)']
    data_to_save['Skin_Conductance_SMA(60s)'] = all_data['Skin_Conductance_SMA(60.0s)']
    data_to_save['Skin_Conductance_EWMA(0.01)'] = all_data['Skin_Conductance_EWMA(0.01)']
    data_to_save['Skin_Conductance_EWMA(0.001)'] = all_data['Skin_Conductance_EWMA(0.001)']
    data_to_save['Skin_Conductance_EWMA(0.0001)'] = all_data['Skin_Conductance_EWMA(0.0001)']

    data_to_save["Skin_Conductance_STD"] = all_data["Skin_Conductance_STD"]
    data_to_save["Skin_Conductance_STD_DIFF(1)"] = all_data["Skin_Conductance_STD_DIFF(1)"]
    
    data_to_save['Skin_Conductance_STD_SMA(1s)'] = all_data['Skin_Conductance_STD_SMA(1.0s)']
    data_to_save['Skin_Conductance_STD_SMA(3s)'] = all_data['Skin_Conductance_STD_SMA(3.0s)']
    data_to_save['Skin_Conductance_STD_SMA(60s)'] = all_data['Skin_Conductance_STD_SMA(60.0s)']
    data_to_save['Skin_Conductance_STD_EWMA(0.01)'] = all_data['Skin_Conductance_STD_EWMA(0.01)']
    data_to_save['Skin_Conductance_STD_EWMA(0.001)'] = all_data['Skin_Conductance_STD_EWMA(0.001)']
    data_to_save['Skin_Conductance_STD_EWMA(0.0001)'] = all_data['Skin_Conductance_STD_EWMA(0.0001)']
    
    data_to_save["Skin_Conductance_NORM"] = all_data["Skin_Conductance_NORM"]
    data_to_save["Skin_Conductance_NORM_DIFF(1)"] = all_data["Skin_Conductance_NORM_DIFF(1)"]
    
    data_to_save["Skin_Conductance_NORM_SMA(1s)"] = all_data["Skin_Conductance_NORM_SMA(1.0s)"]
    data_to_save["Skin_Conductance_NORM_SMA(3s)"] = all_data["Skin_Conductance_NORM_SMA(3.0s)"]
    data_to_save['Skin_Conductance_NORM_SMA(60s)'] = all_data['Skin_Conductance_NORM_SMA(60.0s)']
    data_to_save['Skin_Conductance_NORM_EWMA(0.01)'] = all_data['Skin_Conductance_NORM_EWMA(0.01)']
    data_to_save['Skin_Conductance_NORM_EWMA(0.001)'] = all_data['Skin_Conductance_NORM_EWMA(0.001)']
    data_to_save['Skin_Conductance_NORM_EWMA(0.0001)'] = all_data['Skin_Conductance_NORM_EWMA(0.0001)']
    
    data_to_save["Skin_Resistance[kOhms]"] = all_data["Skin_Resistance"]
    data_to_save["Skin_Resistance_DIFF(1)"] = all_data["Skin_Resistance_DIFF(1)"]
    
    data_to_save["Skin_Resistance_SMA(1s)"] = all_data["Skin_Resistance_SMA(1.0s)"]
    data_to_save["Skin_Resistance_SMA(3s)"] = all_data["Skin_Resistance_SMA(3.0s)"]
    data_to_save["Skin_Resistance_SMA(60s)"] = all_data["Skin_Resistance_SMA(60.0s)"]
    data_to_save['Skin_Resistance_EWMA(0.01)'] = all_data['Skin_Resistance_EWMA(0.01)']
    data_to_save['Skin_Resistance_EWMA(0.001)'] = all_data['Skin_Resistance_EWMA(0.001)']
    data_to_save['Skin_Resistance_EWMA(0.0001)'] = all_data['Skin_Resistance_EWMA(0.0001)']
    
    data_to_save["Skin_Resistance_STD"] = all_data["Skin_Resistance_STD"]
    data_to_save["Skin_Resistance_STD_DIFF(1)"] = all_data["Skin_Resistance_STD_DIFF(1)"]
    
    data_to_save["Skin_Resistance_STD_SMA(1s)"] = all_data["Skin_Resistance_STD_SMA(1.0s)"]    
    data_to_save["Skin_Resistance_STD_SMA(3s)"] = all_data["Skin_Resistance_STD_SMA(3.0s)"]   
    data_to_save["Skin_Resistance_STD_SMA(60s)"] = all_data["Skin_Resistance_STD_SMA(60.0s)"]
    data_to_save['Skin_Resistance_STD_EWMA(0.01)'] = all_data['Skin_Resistance_STD_EWMA(0.01)']
    data_to_save['Skin_Resistance_STD_EWMA(0.001)'] = all_data['Skin_Resistance_STD_EWMA(0.001)']
    data_to_save['Skin_Resistance_STD_EWMA(0.0001)'] = all_data['Skin_Resistance_STD_EWMA(0.0001)']
    
    data_to_save["Skin_Resistance_NORM"] = all_data["Skin_Resistance_NORM"]
    data_to_save["Skin_Resistance_NORM_DIFF(1)"] = all_data["Skin_Resistance_NORM_DIFF(1)"]
    
    data_to_save["Skin_Resistance_NORM_SMA(1s)"] = all_data["Skin_Resistance_NORM_SMA(1.0s)"]    
    data_to_save["Skin_Resistance_NORM_SMA(3s)"] = all_data["Skin_Resistance_NORM_SMA(3.0s)"]    
    data_to_save["Skin_Resistance_NORM_SMA(60s)"] = all_data["Skin_Resistance_NORM_SMA(60.0s)"]
    data_to_save['Skin_Resistance_NORM_EWMA(0.01)'] = all_data['Skin_Resistance_NORM_EWMA(0.01)']
    data_to_save['Skin_Resistance_NORM_EWMA(0.001)'] = all_data['Skin_Resistance_NORM_EWMA(0.001)']
    data_to_save['Skin_Resistance_NORM_EWMA(0.0001)'] = all_data['Skin_Resistance_NORM_EWMA(0.0001)']
    
    data_to_save["Skin_Conductance_Phasic_CVX"] = all_data["Skin_Conductance_Phasic_CVX"]
    data_to_save["Skin_Conductance_Phasic_CVX_SMA(1s)"] = all_data["Skin_Conductance_Phasic_CVX_SMA(1.0s)"]
    data_to_save["Skin_Conductance_Phasic_CVX_SMA(3s)"] = all_data["Skin_Conductance_Phasic_CVX_SMA(3.0s)"]
    data_to_save["Skin_Conductance_Phasic_CVX_SMA(60s)"] = all_data["Skin_Conductance_Phasic_CVX_SMA(60.0s)"]
    data_to_save['Skin_Conductance_Phasic_CVX_EWMA(0.01)'] = all_data['Skin_Conductance_Phasic_CVX_EWMA(0.01)']
    data_to_save['Skin_Conductance_Phasic_CVX_EWMA(0.001)'] = all_data['Skin_Conductance_Phasic_CVX_EWMA(0.001)']
    data_to_save['Skin_Conductance_Phasic_CVX_EWMA(0.0001)'] = all_data['Skin_Conductance_Phasic_CVX_EWMA(0.0001)']
    
    data_to_save["Skin_Conductance_Tonic_CVX"] = all_data["Skin_Conductance_Tonic_CVX"]
    
    data_to_save["HR_CONSENSYS[BPM]"] = all_data["PPG_to_HR_CONSENSYS"]
    
    data_to_save["HR_CONSENSYS_SMA(1s)"] = all_data["PPG_to_HR_CONSENSYS_SMA(1.0s)"]
    data_to_save["HR_CONSENSYS_SMA(3s)"] = all_data["PPG_to_HR_CONSENSYS_SMA(3.0s)"]
    data_to_save["HR_CONSENSYS_SMA(60s)"] = all_data["PPG_to_HR_CONSENSYS_SMA(60.0s)"]
    data_to_save['HR_CONSENSYS_EWMA(0.01)'] = all_data['PPG_to_HR_CONSENSYS_EWMA(0.01)']
    data_to_save['HR_CONSENSYS_EWMA(0.001)'] = all_data['PPG_to_HR_CONSENSYS_EWMA(0.001)']
    data_to_save['HR_CONSENSYS_EWMA(0.0001)'] = all_data['PPG_to_HR_CONSENSYS_EWMA(0.0001)']
    
    
    data_to_save["HR_CONSENSYS_interpolated[BPM]"] = all_data["HR_CONSENSYS_interpolated"]
    data_to_save["HR_CONSENSYS_interpolated_DIFF(1)"] = all_data["HR_CONSENSYS_interpolated_DIFF(1)"]
    
    data_to_save["HR_CONSENSYS_interpolated_SMA(1s)"] = all_data["HR_CONSENSYS_interpolated_SMA(1.0s)"]
    data_to_save["HR_CONSENSYS_interpolated_SMA(3s)"] = all_data["HR_CONSENSYS_interpolated_SMA(3.0s)"]
    data_to_save["HR_CONSENSYS_interpolated_SMA(60s)"] = all_data["HR_CONSENSYS_interpolated_SMA(60.0s)"]
    data_to_save['HR_CONSENSYS_interpolated_EWMA(0.01)'] = all_data['HR_CONSENSYS_interpolated_EWMA(0.01)']
    data_to_save['HR_CONSENSYS_interpolated_EWMA(0.001)'] = all_data['HR_CONSENSYS_interpolated_EWMA(0.001)']
    data_to_save['HR_CONSENSYS_interpolated_EWMA(0.0001)'] = all_data['HR_CONSENSYS_interpolated_EWMA(0.0001)']
    
    data_to_save["HR_CONSENSYS_interpolated_STD"] = all_data["HR_CONSENSYS_STD"]
    data_to_save["HR_CONSENSYS_interpolated_STD_DIFF(1)"] = all_data["HR_CONSENSYS_STD_DIFF(1)"]
    
    data_to_save["HR_CONSENSYS_interpolated_STD_SMA(1s)"] = all_data["HR_CONSENSYS_STD_SMA(1.0s)"]
    data_to_save["HR_CONSENSYS_interpolated_STD_SMA(3s)"] = all_data["HR_CONSENSYS_STD_SMA(3.0s)"]
    data_to_save["HR_CONSENSYS_interpolated_STD_SMA(60s)"] = all_data["HR_CONSENSYS_STD_SMA(60.0s)"]
    data_to_save['HR_CONSENSYS_interpolated_STD_EWMA(0.01)'] = all_data['HR_CONSENSYS_STD_EWMA(0.01)']
    data_to_save['HR_CONSENSYS_interpolated_STD_EWMA(0.001)'] = all_data['HR_CONSENSYS_STD_EWMA(0.001)']
    data_to_save['HR_CONSENSYS_interpolated_STD_EWMA(0.0001)'] = all_data['HR_CONSENSYS_STD_EWMA(0.0001)']
    
    data_to_save["HR_CONSENSYS_interpolated_NORM"] = all_data["HR_CONSENSYS_NORM"]
    data_to_save["HR_CONSENSYS_interpolated_NORM_DIFF(1)"] = all_data["HR_CONSENSYS_NORM_DIFF(1)"]
    
    data_to_save["HR_CONSENSYS_interpolated_NORM_SMA(1s)"] = all_data["HR_CONSENSYS_NORM_SMA(1.0s)"]
    data_to_save["HR_CONSENSYS_interpolated_NORM_SMA(3s)"] = all_data["HR_CONSENSYS_NORM_SMA(3.0s)"]   
    data_to_save["HR_CONSENSYS_interpolated_NORM_SMA(60s)"] = all_data["HR_CONSENSYS_NORM_SMA(60.0s)"] 
    data_to_save['HR_CONSENSYS_interpolated_NORM_EWMA(0.01)'] = all_data['HR_CONSENSYS_NORM_EWMA(0.01)']
    data_to_save['HR_CONSENSYS_interpolated_NORM_EWMA(0.001)'] = all_data['HR_CONSENSYS_NORM_EWMA(0.001)']
    data_to_save['HR_CONSENSYS_interpolated_NORM_EWMA(0.0001)'] = all_data['HR_CONSENSYS_NORM_EWMA(0.0001)']
    
    data_to_save["IBI_CONSENSYS[ms]"] = all_data['PPG_IBI']
    data_to_save["IBI_CONSENSYS_interpolated[ms]"] = all_data['IBI_CONSENSYS_interpolated']

    data_to_save["HR_HeartPy[BPM]"] = all_data["bpm"]
    data_to_save["HR_HeartPy_SMA(1s)"] = all_data['bpm_SMA(1.0s)']
    data_to_save["HR_HeartPy_SMA(3s)"] = all_data["bpm_SMA(3.0s)"]
    data_to_save["HR_HeartPy_SMA(60s)"] = all_data['bpm_SMA(60.0s)']
    data_to_save['HR_HeartPy_EWMA(0.01)'] = all_data['bpm_EWMA(0.01)']
    data_to_save['HR_HeartPy_EWMA(0.001)'] = all_data['bpm_EWMA(0.001)']
    data_to_save['HR_HeartPy_EWMA(0.0001)'] = all_data['bpm_EWMA(0.0001)']
    
    data_to_save["HR_HeartPy_interpolated[BPM]"] = all_data["HR_HeartPy_interpolated"]
    data_to_save["HR_HeartPy_interpolated_DIFF(1)"] = all_data["HR_HeartPy_interpolated_DIFF(1)"]
    
    data_to_save["HR_HeartPy_interpolated_SMA(1s)"] = all_data["HR_HeartPy_interpolated_SMA(1.0s)"]
    data_to_save["HR_HeartPy_interpolated_SMA(3s)"] = all_data["HR_HeartPy_interpolated_SMA(3.0s)"]
    data_to_save["HR_HeartPy_interpolated_SMA(60s)"] = all_data['HR_HeartPy_interpolated_SMA(60.0s)']
    data_to_save['HR_HeartPy_interpolated_EWMA(0.01)'] = all_data['HR_HeartPy_interpolated_EWMA(0.01)']
    data_to_save['HR_HeartPy_interpolated_EWMA(0.001)'] = all_data['HR_HeartPy_interpolated_EWMA(0.001)']
    data_to_save['HR_HeartPy_interpolated_EWMA(0.0001)'] = all_data['HR_HeartPy_interpolated_EWMA(0.0001)']
    
    data_to_save["HR_HeartPy_interpolated_STD"] = all_data["HR_HP_STD"]
    data_to_save["HR_HeartPy_interpolated_STD_DIFF(1)"] = all_data["HR_HP_STD_DIFF(1)"]
    
    data_to_save["HR_HeartPy_interpolated_STD_SMA(1s)"] = all_data["HR_HP_STD_SMA(1.0s)"]
    data_to_save["HR_HeartPy_interpolated_STD_SMA(3s)"] = all_data["HR_HP_STD_SMA(3.0s)"]
    data_to_save["HR_HeartPy_interpolated_STD_SMA(60s)"] = all_data['HR_HP_STD_SMA(60.0s)']
    data_to_save['HR_HeartPy_interpolated_STD_EWMA(0.01)'] = all_data['HR_HP_STD_EWMA(0.01)']
    data_to_save['HR_HeartPy_interpolated_STD_EWMA(0.001)'] = all_data['HR_HP_STD_EWMA(0.001)']
    data_to_save['HR_HeartPy_interpolated_STD_EWMA(0.0001)'] = all_data['HR_HP_STD_EWMA(0.0001)']
    
    data_to_save["HR_HeartPy_interpolated_NORM"] = all_data["HR_HP_NORM"]
    data_to_save["HR_HeartPy_interpolated_NORM_DIFF(1)"] = all_data["HR_HP_NORM_DIFF(1)"]
    
    data_to_save["HR_HeartPy_interpolated_NORM_SMA(1s)"] = all_data["HR_HP_NORM_SMA(1.0s)"]
    data_to_save["HR_HeartPy_interpolated_NORM_SMA(3s)"] = all_data["HR_HP_NORM_SMA(3.0s)"]
    data_to_save["HR_HeartPy_interpolated_NORM_SMA(60s)"] = all_data['HR_HP_NORM_SMA(60.0s)']
    data_to_save['HR_HeartPy_interpolated_NORM_EWMA(0.01)'] = all_data['HR_HP_NORM_EWMA(0.01)']
    data_to_save['HR_HeartPy_interpolated_NORM_EWMA(0.001)'] = all_data['HR_HP_NORM_EWMA(0.001)']
    data_to_save['HR_HeartPy_interpolated_NORM_EWMA(0.0001)'] = all_data['HR_HP_NORM_EWMA(0.0001)']    
    
    data_to_save["IBI_HeartPy[ms]"] = all_data["ibi"]
    data_to_save["IBI_HeartPy_interpolated[ms]"] = all_data["IBI_HeartPy_interpolated"]
    
    data_to_save["SDNN_HeartPy[ms]"] = all_data["sdnn"]
    data_to_save["SDNN_HeartPy_interpolated[ms]"] = all_data["SDNN_HeartPy_interpolated"]
    
    data_to_save["SDSD_HeartPy[ms]"] = all_data["sdsd"]
    data_to_save["SDSD_HeartPy_interpolated[ms]"] = all_data["SDSD_HeartPy_interpolated"]
    
    data_to_save["RMSSD_HeartPy[ms]"] = all_data["rmssd"]
    data_to_save["RMSSD_HeartPy_interpolated[ms]"] = all_data["RMSSD_HeartPy_interpolated"]
    
    data_to_save["pNN20_HeartPy[%]"] = all_data["pnn20"]
    data_to_save["pNN20_HeartPy_interpolated[%]"] = all_data["pNN20_HeartPy_interpolated"]
    
    data_to_save["pNN50_HeartPy[%]"] = all_data["pnn50"]
    data_to_save["pNN50_HeartPy_interpolated[%]"] = all_data["pNN50_HeartPy_interpolated"]
    
    data_to_save["Pressure[kPa]"] = all_data["Pressure"]
    data_to_save["Temperature[C]"] = all_data["Temperature"]
    """
    
    # data_to_save["breathing-rate_HeartPy[?]"] = all_data["breathingrate"] # all values = 0 ?
    
    
    # export to .xlsx takes over twice as long as .csv but reduces the file size by over half: 35MB -> 14MB (partially due to 2nd sheet containing demographic data: approx 4MB)

    data_to_save.index.names = ['INDEX_in_all_data']
    demographicData.index.names = ['PARAMETER']
    demographicData.rename(columns = {0:'VALUE'}, inplace = True)
    if(save):
        try:
            filePath = "MERGED_DATA\merged_" + str(ID) + "-" + str(GR) + ".xlsx"
            with pd.ExcelWriter(os.path.join(os.getcwd(),filePath)) as writer:
                demographicData.to_excel(writer, sheet_name='Participant_Data')
                data_to_save.to_excel(writer, sheet_name='Experiment_Data')
        except Exception as e:
            print("error_2__1 : exporting raw file path?")
            print(e)

    # added temp time to avoid string parsing to datetime 
    
    # data_to_save['TEMP_Time'] = all_data['TEMP_Time']
    
    return data_to_save

## *** Clean up ***##

In [27]:
def removeTails(data, GR, testMode = False):
    if testMode:
        print("START removeTails----------------")
        print(f"len(data) : {len(data)}")
        
    phase = ""
    endIndex = 0
    
    if(GR == 1):
        phase = 'control'
    else:
        phase = 'intervention'
    
    endIndex = data['Phase'][data['Phase'] == phase].index[-1]
    
    if testMode:
        print(f"------------endIndex : {endIndex} ----------")
        print(data['Phase'].iloc[endIndex -1 : endIndex + 2])    
        print(data.iloc[endIndex -1 : endIndex + 2])
        
    data = data[:endIndex]
    
    if testMode:
        print("new tail======")
        print(data.tail(3))
        print(f"len(data) : {len(data)}")
    
    # front clean up moved to end to avoid indexing issues: 
    
    startIndex = data['Phase'][data['Phase'] == 'adaptation PE'].index[0]
    
    if testMode:
        print(f"------------endIndex : {startIndex} ----------")
        print(data['Phase'].iloc[startIndex -2 : startIndex + 3])
    
    data = data[startIndex:]
    
    if testMode:
        print("new head======")
        print(data.head(3))
        print(f"len(data) : {len(data)}")
    
    data.rename(columns = {'index':'index_in_all_data'}, inplace = True)    
    data = data.reset_index()
    
    if testMode:
        print("Final product returned : ")
        print(data)
        print("END----------------")
    
    """
    removed abs time from the data set
    
    timeToSubstract = data["TEMP_Time"][0]   
    absolute_time = data['TEMP_Time'] - timeToSubstract

    location = data.columns.get_loc("Raw_Time[HH:MM:ss.us]")
    data.insert(loc=location, column='Absolute_time[HH:MM:ss.us]', value = absolute_time)
    
    data.drop(columns = 'TEMP_Time', inplace = True)
    """
    
    return data

In [28]:
def removeBreak(data):
    """
    was used to remove the break
    left for now to use it for processing break stage later
    """
    
    breakIndexes = np.argwhere(data['Phase'] == 'break')
    firstBreakInd = breakIndexes[0]
    lastBreakInd = breakIndexes[-1]
    
    firstBreakTimeStamp = data['Absolute_time[HH:MM:ss.us]'].iloc[firstBreakInd].values
    timeStampFirst = pd.to_datetime(firstBreakTimeStamp) 

    lastBreakTimeStamp = data['Absolute_time[HH:MM:ss.us]'].iloc[lastBreakInd].values
    timeStampLast = pd.to_datetime(lastBreakTimeStamp)
    
    breakDuration = pd.Timedelta(timeStampLast[0] - timeStampFirst[0])

    start = lastBreakInd + 1 
    end = len(data)
    TIME = data['Absolute_time[HH:MM:ss.us]']
    indexesToCorrect =  np.arange(start, end)
    
    TIME[indexesToCorrect] = TIME[indexesToCorrect] - breakDuration 

    data['Absolute_time[HH:MM:ss.us]'] = TIME
    
    data.drop(data.index[breakIndexes], inplace = True)

    # data.rename(columns = {'index':'index_no_tails'}, inplace = True) 
    data = data.reset_index()
    
    return data

Resample data to 100Hz and merge asof

In [29]:
def standardizeTime(data):
    """
    Deprecated since break phase was uneven length
    """
    
    timeDF = pd.DataFrame()
    
    TotalTime = 2*90 + 16*12*2 # adaptationPE = 90s; adaptationVE = 90s; intervention = 12s * 16; control = intervention
    numberOfSamples = TotalTime * 100 # to 100Hz
    
    #timeDF['TIME'] = pd.timedelta_range(0, periods = numberOfSamples, freq="10L")
    #timeDF['Absolute_time[HH:MM:ss.us]'] = timeDF['TIME'].values.to_datetime()
    
    timeDF['TIME[HH:MM:ss.us]'] = pd.timedelta_range(0, periods = numberOfSamples, freq="10L")        
    timeDF['TIME[HH:MM:ss.us]'] = pd.to_datetime(timeDF['TIME[HH:MM:ss.us]'])

    data['TIME[HH:MM:ss.us]'] = pd.to_datetime(data['Absolute_time[HH:MM:ss.us]'])
    
    all_data = pd.merge_asof(timeDF, data, on='TIME[HH:MM:ss.us]', direction='nearest', tolerance = pd.Timedelta('50ms'))
    
    return all_data

In [30]:
def saveCleanedData(ID, GR, data, demographicData, testMode = False, save = False):
    save = saveGlobal
    #belowed removed as abs time deprecated
    #data.drop(columns = 'Absolute_time[HH:MM:ss.us]', inplace = True)
    data.drop(columns = 'index', inplace = True)
    
    #timeDF['TIME'] = pd.timedelta_range(0, periods = numberOfSamples, freq="10L")
    #timeDF['Absolute_time[HH:MM:ss.us]'] = timeDF['TIME'].values.to_datetime()
    
    if(testMode):
        print("===================data['TIME']===============")
        print(data['TIME'])
    
    #I'm not even sure if its needed
    #data['TIME[HH:MM:ss.us]'] = data['TIME'].values.to_datetime().dt.strftime('%H:%M:%S.%f')
    
    data['TIME[HH:MM:ss.us]'] = pd.to_datetime(data['TIME'],format = '%H:%M:%S.%f')
    
    if(save):
        filePath = "MERGED_DATA/Cleaned/" + str(ID) + "-" + str(GR) + ".xlsx"
        with pd.ExcelWriter(filePath) as writer:
            demographicData.to_excel(writer, sheet_name='Participant_Data')
            data.to_excel(writer, sheet_name='Experiment_Data')

## 5? try to go through all functions in a sequence

In [31]:
def executeAllFunctions(item, save = True, testMode = False):
    testMode = testModeGlobal
    save = saveGlobal
    #print(f"testMode {testMode}")
    
    if testMode:
        print("executeAllFunctions======")
        print(item)
        print(type(item))
        print(item[0])
        print(type(item[0]))
        print(item[1])
        print(type(item[1]))
        
    ID = item[0]
    GR = item[1]
    
    print(f"ID = {ID} | GR = {GR}")
    
    #global participantID
    #participantID = ID
    #global participantGR
    #participantGR = GR
    
    goTime = time.time()
    
    try:
        environment_data = pullEnvironmentData(ID, GR)
        #print(environment_data.head(1))
    except Exception as e:
        print("error_2 : env data - import <- pullEnvironmentData()")
        print(e)    
    if testMode:
        print("pullEnvironmentData() - completed")
    
    try:
        participant_data = pullPhysioData(ID, GR)
        #print(participant_data.head(1))
    except Exception as e: 
        print("error_2 : participant data <- pullPhysioData()")
        print(e)
    if testMode:
        print("pullPhysioData - completed") 
    
    try:    
        qualityScore = qualityCheck(participant_data)
    except Exception as e: 
        print("error_2 : participant data - testing quality")
        print(e)
    if testMode:
        print("qualityCheck - completed")  
            
    try:
        participant_data = interpolateConsensysHRna(participant_data)
    except Exception as e: 
        print("error_2 : participant data - interpolate NA")
        print(e)
    if testMode:
        print("interpolateConsensysHRna - completed")  

    TS = [timeStamp.strftime('%Y-%m-%d %H:%M:%S.%f') for timeStamp in participant_data['timestamp']]
    sampleRate = hp.get_samplerate_datetime(TS, timeformat = '%Y-%m-%d %H:%M:%S.%f')
    
    #ERROR BELOW
    """
    try:
        savePlot(participant_data["PPG_A13"].values, sampleRate, winSize, scale)
    except Exception as e: 
        print("error_2 : HP peak graph")
        print(e)
    if testMode:
        print("savePlot - completed")  
    """
    try:
        period = getPeriod(participant_data["PPG_A13"].values, sampleRate, winSize, overLap)
    except Exception as e:
        print("error_2 : HP period")
        print(e)  
    if testMode:
        print("getPeriod - completed")  

    try:
        hp_data = getContDataFromHP(participant_data["PPG_A13"].values, sampleRate, winSize, overLap, scale, period)
    except Exception as e:
        print("error_2 : HP continous")
        print(e)
    if testMode:
        print("getContDataFromHP - completed")  
    
    """
    #some bool ?? cannot be executed here, some error in saveHRplot 
    try:
        saveHRplot(ID, GR, hp_data, participant_data)
    except Exception as e:
        print("error_2 : HP HR graph")
        print(e)
    if testMode:
        print("saveHRplot - completed")  
    """
    
    try:
        demographicData = pullDemoQdata(ID,GR)
    except Exception as e:
        print("error_2 : demographic data - import")
        print(e)
    if testMode:
        print("pullDemoQdata - completed")
    
    try:
        try:
            participant_data = mergeData(participant_data, hp_data)
        except Exception as e:
            print("error_2_1 : merging")
            print(e)
        if testMode:
            print("mergeData - completed") 
        
        try:
            all_data = mergeAll(participant_data, environment_data)
        except Exception as e:
            print("error_2_2 : merging")
            print(e)
        if testMode:
            print("mergeAll - completed") 
        
        """
        absoluteTime function deprecated
        
        try:
            all_data = absoluteTime(all_data)
        except Exception as e:
            print("error_2_3 : merging")
            print(e)
        print("absoluteTime - completed")
        """
        
    except Exception as e:
        print("error_2 : merging")
        print(e)
    if testMode:
        print("merging - completed")  
    
    try: 
        all_data = standardizeData(all_data)
    except Exception as e:
        print("error_2 : standardizing data")
        print(e)
    if testMode:
        print("standardizeData - completed")  
    
    try: 
        all_data = normalizeData(all_data)
    except Exception as e:
        print("error_2 : normalizing data")
        print(e)
    if testMode:
        print("normalizeData - completed")  
        
    try:
        all_data = processEDA(all_data)
    except Exception as e:
        print("error_2 : processing EDA")
        print(e)
    if testMode:
        print("processEDA - completed")  
    
    try:
        all_data = smoothHRandSC(all_data)
    except Exception as e:
        print("error_2 : smoothin MA")
        print(e)
    if testMode:
        print("smoothHRandSC - completed")  
        
    try:
        all_data = smoothHRandSCewma(all_data)
    except Exception as e:    
        print("error_2 : smoothin EWMA")
        print(e)
    if testMode:
        print("smoothHRandSCewma - completed")  
        
    try:
        all_data = diff1st(all_data)
    except Exception as e:
        print("error_2 : differentiating Data")
        print(e)
    if testMode:
        print("diff1st - completed")  
    
    
    #"""
    savingStartTime = time.time()
    try:
        savedData = saveAllData(all_data, ID, GR, demographicData, save=save)
    except Exception as e:
        print("error_2 : exporting raw")
        print(e)
    if testMode:
        print(f"saveAllData - completed in {time.time() - savingStartTime}")  
    
    #"""
    
    try:
        noTailsData = removeTails(all_data, GR = GR, testMode = testMode)#savedData)
    except Exception as e:
        print("error_2 : failed to remove tails")
        print(e)
    if testMode:
        print("removeTails - completed")  
    
    #below removed to keep the break data
    """
    try:
        cleanData = removeBreak(noTailsData)
    except Exception as e:
        print("error_2 : failed to remove break")
        print(e)
    print("removeBreak - completed")  
    """
    
    #belowed removed as break phase lenght was uneven
    """
    try:
        cleanData = standardizeTime(noTailsData) #standardizeTime(cleanData)
    except Exception as e:
        print("error_2 : failed to standardized time data")
        print(e)
    print("standardizeTime - completed")  
    """
    
    try:
        saveCleanedData(ID, GR, noTailsData, demographicData, save = save)#cleanData, demographicData)
    except Exception as e:
        print("error_2 : exporting clean")
        print(e)
    if testMode:
        print("saveCleanedData - completed") 
    
    goTime = time.time() - goTime
    print(f"*********allFunctions - completed ********* runtime = {goTime} **********")
    
    qualityScores.append(qualityScore)
    noTailsDataSet.append(noTailsData)
    demographicDataSet.append(demographicData)
    runTimeSet.append(goTime)
    
    return qualityScore, noTailsData, demographicData, goTime

## ***PROCESS ALL***

In [32]:
def processAll(dictionary, save=True, multi = True, testMode = False, advancedMode = False):
    testMode = testModeGlobal
    save = saveGlobal
    advancedMode = advancedModeGlobal
    
    procedureStartTime = pd.Timestamp.now()
    
    #qualityScores = []
    #noTailsDataSet = []
    #demographicDataSet = []
    #runTimeSet = []
        
    
    def thredExecuteAllFunctions(item, item2):
        try:
            print(f'{item[0]} : {item[1]}')
            print(item2)
            #qualityScore, noTailsData, demographicData, goTime = executeAllFunctions(item, save = save, testMode = testMode)

            
            #??? return qualityScore, noTailsData, demographicData, goTime
            #qualityScore, noTailsData, demographicData, goTime = executeAllFunctions(item, save = save, testMode = testMode)
            executeAllFunctions(item, save = save, testMode = testMode)
            
            # trying changing scope first
            #qualityScores.append(qualityScore)
            #noTailsDataSet.append(noTailsData)
            #demographicDataSet.append(demographicData)
            #runTimeSet.append(goTime)
            
        except Exception as e:
            print(f"fatal error_1 : thredExecuteAllFunctions() in {item}")
            print(e)
            
    def thredExecuteAllFunctionsADVANCE(item):
        try:
            print(f'{item[0]} : {item[1]}')
            executeAllFunctions(item, save = save, testMode = testMode)
            
            #qualityScore, noTailsData, demographicData, goTime = executeAllFunctions(item, save = save, testMode = testMode)
            """
            ---------------------------------------------------------------------------
            TypeError                                 Traceback (most recent call last)
            <ipython-input-38-6b9cfbaea136> in <module>
            ----> 1 qualityScores, noTailsDataSet, demographicDataSet, runTimeSet = processAll(AllParticipantsGr, save = False, testMode = True)

            <ipython-input-32-26328fb9ac4d> in processAll(dictionary, save, multi, testMode)
                 44 
                 45             for process in concurrent.futures.as_completed(results):
            ---> 46                 qualityScores.append(process.result()[0])
                 47                 noTailsDataSet.append(process.result()[1])
                 48                 demographicDataSet.append(process.result()[2])

            TypeError: 'NoneType' object is not subscriptable
            """
            
            #??? return qualityScore, noTailsData, demographicData, goTime
            """
            qualityScore, noTailsData, demographicData, goTime = executeAllFunctions(item, save = save, testMode = testMode)
            
            # trying changing scope first
            qualityScores.append(qualityScore)
            noTailsDataSet.append(noTailsData)
            demographicDataSet.append(demographicData)
            runTimeSet.append(goTime)
            """
        except Exception as e:
            print(f"fatal error_1 : thredExecuteAllFunctionsADVANCE() in {item}")
            print(e)
    
    
    #multiprocessing
    if multi and advancedMode:
        print(f"advanced mode : {advancedMode} | multiprocessing : {multi}")
        
        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = [executor.submit(thredExecuteAllFunctionsADVANCE, i) for i in list(dictionary.items())]
            
            for process in concurrent.futures.as_completed(results):
                print(process)
    
    elif multi:
        print(f"advanced mode : {advancedMode} | multiprocessing : {multi}")
        try:
            threads = []

            for i in list(dictionary.items()):
                print(i)
                print(type(i))
                t = threading.Thread(target = executeAllFunctions, args = [i])#args = [1])
                t.start()
                threads.append(t)

            for t in threads:
                t.join()
                
        except Exception as e:
            print("fatal error_1 : executeAllFunctions()")
            print(e)
        
    else:
        print(f"multiprocessing : {multi}")
        try:
            for item in list(dictionary.items()):
                print(f'{item[0]} : {item[1]}')
                executeAllFunctions(item, save = save, testMode = False)
                
                #qualityScore, noTailsData, demographicData, goTime = executeAllFunctions(item, save = save, testMode = False)
                #qualityScores.append(qualityScore)
                #noTailsDataSet.append(noTailsData)
                #demographicDataSet.append(demographicData)
                #runTimeSet.append(goTime)
            
        except Exception as e:
            print("fatal error_1 : executeAllFunctions()")
            print(e)
    
    print(f'Total time to process : {pd.Timestamp.now() - procedureStartTime}')
    
    return qualityScores, noTailsDataSet, demographicDataSet, runTimeSet

# Run:

# 1 : Parameters:  

In [33]:
global qualityScores
qualityScores = []
global noTailsDataSet
noTailsDataSet = []
global demographicDataSet
demographicDataSet = []
global runTimeSet
runTimeSet = []

global testModeGlobal
testModeGlobal = False
global saveGlobal
saveGlobal = True
global multiGlobal
multiGlobal = True
global advancedModeGlobal
advancedModeGlobal = True

### 1.1 : Participants list (dictionary key: id | value: group)

In [34]:
AllParticipantsGr = {'0002':2}

In [35]:
AllParticipantsGr = {'0048':1,'0053':2,'0061':2,'0071':2}

In [36]:
AllParticipantsGr = {'0002':2,'0003':1,'0004':2,'0005':2,'0006':1,'0007':1,'0008':1,'0009':2,'0010':2,
                     '0011':2,'0012':2,'0013':2,'0014':1,'0015':2,'0016':1,'0017':2,'0018':2,'0019':2,'0020':1,
                     '0021':2,'0022':1,'0023':1,'0024':1,'0025':1,'0026':1,'0027':1,'0028':1,'0029':2,'0030':1,
                     '0032':2,'0033':2,'0034':2,'0035':1,'0036':2,'0037':1,'0038':1,'0039':2,'0040':2,
                     '0041':2,'0042':2,'0043':1,'0044':2,'0045':1,'0046':1,'0047':1,'0048':1,'0049':2,'0050':1,
                     '0051':1,'0052':1,'0053':2,'0054':2,'0055':2,'0056':1,'0057':1,'0058':1,'0059':1,'0060':2,
                     '0061':2,'0062':2,'0063':1,
                     '0071':2,'0072':1,'0073':2
                    }

In [37]:
print(len(AllParticipantsGr))

64


### 1.2 : Run parameters

In [38]:
winSize = 6 # in seconds
overLap = 0.5 # % overlap
sampleRate = 128.0
scale = True

### 1.3 : Process all data

In [ ]:
qualityScores, noTailsDataSet, demographicDataSet, runTimeSet = processAll(AllParticipantsGr, save = False, testMode = True, multi = True)

advanced mode : True | multiprocessing : True
0002 : 20003 : 1
ID = 0002 | GR = 2

ID = 0003 | GR = 1
0004 : 2
ID = 0004 | GR = 2
0005 : 2
ID = 0005 | GR = 2
0006 : 1
ID = 0006 | GR = 1
0007 : 1
ID = 0007 | GR = 1
0008 : 1
ID = 0008 | GR = 1
0009 : 2
ID = 0009 | GR = 2
0010 : 2
ID = 0010 | GR = 20011 : 2

ID = 0011 | GR = 2
0012 : 2
ID = 0012 | GR = 2
0013 : 2
ID = 0013 | GR = 20014 : 1

ID = 0014 | GR = 10015 : 2

ID = 0015 | GR = 2
0016 : 1
0017 : 2
ID = 0017 | GR = 2
ID = 0016 | GR = 10018 : 2

ID = 0018 | GR = 20019 : 2
ID = 0019 | GR = 2

0020 : 1
ID = 0020 | GR = 1
0021 : 2
ID = 0021 | GR = 2
0022 : 1
ID = 0022 | GR = 1
0023 : 1
0024 : 1ID = 0023 | GR = 1

0025 : 1ID = 0024 | GR = 1

ID = 0025 | GR = 1
0026 : 1
ID = 0026 | GR = 1
0027 : 1
ID = 0027 | GR = 1
0028 : 1
ID = 0028 | GR = 1
0029 : 2
0030 : 1
ID = 0030 | GR = 1
0032 : 2
ID = 0032 | GR = 2
ID = 0029 | GR = 20033 : 2

ID = 0033 | GR = 20034 : 2

ID = 0034 | GR = 2


C:\Users\Krzyniu\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Krzyniu\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Krzyniu\anaconda3\lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Krzyniu\anaconda3\lib\site-packages\numpy\core\_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Krzyniu\anaconda3\lib\site-packages\numpy\core\_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Krzyniu\anaconda3\lib\site-packages\numpy\core\_methods.py:180: RuntimeWarning: invalid value encount

     pcost       dcost       gap    pres   dres
     pcost       dcost       gap    pres   dres     pcost       dcost       gap    pres   dres

     pcost       dcost       gap    pres   dres
 0: -3.9260e+04 -3.8983e+04  1e+05  4e+02  9e+00 0: -4.0100e+04 -4.0017e+04  1e+05  3e+02  6e+00

 0: -3.5660e+04 -3.3138e+04  4e+05  6e+02  2e+01 0: -3.9857e+04 -3.9512e+04  1e+05  3e+02  7e+00

 1: -4.0041e+04 -4.6913e+04  7e+03  2e+01  5e-01 1: -3.9318e+04 -5.4033e+04  2e+04  5e+01  1e+00

 1: -3.7386e+04 -1.1748e+05  1e+05  2e+02  4e+00
 1: -3.9965e+04 -5.3541e+04  2e+04  4e+01  9e-01
 2: -4.0038e+04 -4.1423e+04  1e+03  4e+00  7e-02
 2: -3.9385e+04 -4.2474e+04  3e+03  7e+00  2e-01
 2: -3.8525e+04 -9.4364e+04  6e+04  7e+01  2e+00
 2: -4.0048e+04 -4.2119e+04  2e+03  5e+00  1e-01
 3: -4.0044e+04 -4.0313e+04  3e+02  6e-01  1e-02
 3: -3.9392e+04 -3.9979e+04  6e+02  1e+00  3e-02
 3: -3.8807e+04 -5.9348e+04  2e+04  2e+01  6e-01
 4: -4.0052e+04 -4.0148e+04  1e+02  8e-02  2e-03 3: -4.0060e+04 -4.0495e+

<ipython-input-21-7d090e579a5c>:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  NewDF[columnName] = data[parameter].ewm(alpha = alpha).mean()


 8: -3.9431e+04 -3.9452e+04  2e+01  4e-03  2e-04
 9: -3.9440e+04 -3.9449e+04  9e+00  2e-03  6e-05
10: -3.9443e+04 -3.9447e+04  4e+00  7e-04  2e-05
11: -3.9445e+04 -3.9447e+04  1e+00  2e-04  6e-06


...

<ipython-input-27-0383d79aeb3c>:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data = data.reset_index()


     pcost       dcost       gap    pres   dres
 0: -4.3031e+04 -4.2956e+04  9e+04  3e+02  5e+00
 1: -4.2962e+04 -4.5435e+04  3e+03  8e+00  1e-01
 2: -4.2966e+04 -4.3254e+04  3e+02  8e-01  1e-02
 3: -4.2981e+04 -4.3037e+04  6e+01  8e-03  1e-04
 4: -4.3015e+04 -4.3035e+04  2e+01  3e-03  4e-05
 5: -4.3022e+04 -4.3035e+04  1e+01  1e-03  2e-05
 6: -4.3029e+04 -4.3035e+04  6e+00  6e-04  9e-06
 7: -4.3033e+04 -4.3034e+04  2e+00  1e-04  2e-06
 8: -4.3033e+04 -4.3034e+04  1e+00  6e-05  1e-06
 9: -4.3034e+04 -4.3034e+04  3e-01  1e-05  2e-07
10: -4.3034e+04 -4.3034e+04  2e-01  8e-06  1e-07
11: -4.3034e+04 -4.3034e+04  6e-02  2e-06  3e-08
12: -4.3034e+04 -4.3034e+04  3e-02  4e-07  7e-09
13: -4.3034e+04 -4.3034e+04  7e-03  7e-08  1e-09
Optimal solution found.


<ipython-input-21-7d090e579a5c>:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  NewDF[columnName] = data[parameter].ewm(alpha = alpha).mean()
<ipython-input-22-faa363d1e944>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[columnName] = diffData
<ipython-input-26-56396ebd70d1>:141: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_to_save["HR_CONSENSYS_interpolated_NORM_SMA(60s)"] = all_data["H

<ipython-input-26-56396ebd70d1>:167: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_to_save["HR_HeartPy_interpolated_STD"] = all_data["HR_HP_STD"].copy()
<ipython-input-26-56396ebd70d1>:168: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_to_save["HR_HeartPy_interpolated_STD_DIFF(1)"] = all_data["HR_HP_STD_DIFF(1)"].copy()
<ipython-input-26-56396ebd70d1>:170: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.c

<ipython-input-26-56396ebd70d1>:200: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_to_save["pNN20_HeartPy_interpolated[%]"] = all_data["pNN20_HeartPy_interpolated"].copy()
<ipython-input-26-56396ebd70d1>:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_to_save["pNN50_HeartPy[%]"] = all_data["pnn50"].copy()
<ipython-input-26-56396ebd70d1>:203: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dat

In [ ]:
print(len(qualityScores))
print(qualityScores)
print(f"average quality score = {sum(qualityScores)/len(qualityScores)}")

In [ ]:
plt.hist(qualityScores,64)
plt.ylabel('number of samples')
plt.xlabel('missing HR data %')
plt.show()

In [ ]:
def percentageCalcuator(below, listToCheck):
    sumOfBelow = len([x for x in listToCheck if x < below])
    totalN = len(listToCheck)
    print(f"{sumOfBelow} samples with errors below {below}% out of {totalN} samples = {sumOfBelow/totalN * 100}% of the sample")

In [ ]:
for i in [0, .5, 1, 3, 5, 10, 20, 25, 50, 75, 80, 90]:
    percentageCalcuator(i, qualityScores)

In [ ]:
print(noTailsDataSet) 

In [ ]:
print(len(demographicDataSet))
print(demographicDataSet)

In [ ]:
print(runTimeSet)

In [ ]:
def runTest():
    participantID = '0002'
    participantGR = 2
    #Virtual Enviornment Data
    environment_data = pullEnvironmentData(participantID,participantGR)
    print(environment_data.head())
    physiologicaData = pullPhysioData(participantID,participantGR)
    print(physiologicaData.head())
    #return environment_data

In [ ]:
#runTest()

In [ ]:
print(f"total processing time = {runTimeSet}")

In [ ]:
print(f"average processing time = {sum(runTimeSet)/len(runTimeSet)}")

In [ ]:
print(f"average ambiguous HR data = {sum(qualityScores)/len(qualityScores)}")

In [ ]:
columnNames = noTailsDataSet[0].columns

In [ ]:
print(columnNames[5:22])

In [ ]:
noTailsDataSet[0].head()

In [ ]:
breakStarts = [dataFrame['Phase'][dataFrame['Phase'] == 'break'].index[0] for dataFrame in noTailsDataSet]
print(len(breakStarts))
print(breakStarts)
breakEnds = [dataFrame['Phase'][dataFrame['Phase'] == 'break'].index[-1] for dataFrame in noTailsDataSet]
print(breakEnds)

In [ ]:
breakLenght = lambda start, finish: (finish - start)/60
breakLenghts = [breakLenght(key, value) for key, value in zip(breakStarts, breakEnds)]
print(breakLenghts)

In [ ]:
from scipy import stats

In [ ]:
stats.normaltest(breakLenghts)

In [ ]:
plt.hist(breakLenghts,30)

In [ ]:
## Within Subject approach

In [ ]:
def analyzeThis(column):
    print(f"<+>==================={column}+++++++++++++++++++++++<<<")
    pre = [noTailsDataSet[i].loc[:breakStarts[i]-1, column] for i in range(0, len(noTailsDataSet))]
    post = [noTailsDataSet[i].loc[:breakEnds[i], column] for i in range(0, len(noTailsDataSet))]

    meanForTS = lambda timeSeries: sum(timeSeries)/len(timeSeries)
    meansPre = [meanForTS(item) for item in pre]
    meansPost = [meanForTS(item) for item in post]

    print(f"{column} : {stats.shapiro(meansPre)}")
    plt.hist(meansPre,20)
    plt.show()
    
    print(f"{column} : {stats.shapiro(meansPost)}")
    plt.hist(meansPost,20)
    plt.show()

    print(f"{column} : {stats.ttest_rel(meansPre, meansPost)}")
    print(f"{column} : {stats.wilcoxon(meansPre, meansPost)}")

    #print(stats.ttest_rel(meansPre, meansPost, alternative = "greater"))
    #print(stats.wilcoxon(meansPre, meansPost, alternative = "greater"))

In [ ]:
analyzeThis("bpm") #Skin_Resistance

In [ ]:
for column in columnNames:
    try:
        print(column)
        analyzeThis(column)
    except Exception as e:
        print("can't count this!")
        print(e)
    continue

In [ ]:
audioStarts = [dataFrame['Audio_clip'][dataFrame['Audio_clip'] == 'Audio_1'].index[0] for dataFrame in noTailsDataSet]
audioEnds = [dataFrame['Audio_clip'][dataFrame['Audio_clip'] == 'Audio_1'].index[-1] for dataFrame in noTailsDataSet]
   

In [ ]:
print(audioStarts)
print(audioEnds)

In [ ]:
print(demographicDataSet[1].head())

## Drawing all the columns data

In [ ]:
def drawThis(column):
    from bokeh.models.tools import HoverTool    
    
    print(f"<+>==============Drawing={column}+++++++++++++++++++++++<<<")
    """
    print(demographicDataSet[1].head())
    print("==============")
    print(noTailsDataSet[1].head())
    print(pre[1])
    print("==============")
    """
    
    pre = [noTailsDataSet[i].loc[:breakStarts[i]-1, column] for i in range(0, len(noTailsDataSet))]
    instruction = [noTailsDataSet[i].loc[audioStarts[i]:audioEnds[i]-1, column] for i in range(0, len(noTailsDataSet))]
    post = [noTailsDataSet[i].loc[audioEnds[i]:breakEnds[i], column] for i in range(0, len(noTailsDataSet))]
    #postAll = [noTailsDataSet[i].loc[:breakEnds[i], column] for i in range(0, len(noTailsDataSet))]

    #MEANS not needed now
    """
    meanForTS = lambda timeSeries: sum(timeSeries)/len(timeSeries)
    meansPre = [meanForTS(item) for item in pre]
    meansPost = [meanForTS(item) for item in post]

    print(f"{column} : {stats.shapiro(meansPre)}")
    plt.hist(meansPre,20)
    plt.show()
    
    print(f"{column} : {stats.shapiro(meansPost)}")
    plt.hist(meansPost,20)
    plt.show()

    print(f"{column} : {stats.ttest_rel(meansPre, meansPost)}")
    print(f"{column} : {stats.wilcoxon(meansPre, meansPost)}")
    """
    
    ###OTher way through DF
    columnNames = [int(demographicDataSet[i].loc["ID"]) for i in range(0, len(demographicDataSet))]
    
    print(columnNames)
    
    dfPre = pd.concat(pre, axis = 1, keys = columnNames)
    #dfPre = pd.pivot_table(dfPre)
    #print(dfPre) 
    dfInstruction = pd.concat(instruction, axis = 1, keys = columnNames)
    #print(dfInstruction)
    dfPost = pd.concat(post, axis = 1, keys = columnNames)    
    #print(dfInstruction)
    
    graphPath = "Graphs\/"+ column + ".html";
    output_file(graphPath)
    
    TOOLTIPS = [
        ("index", "$index"),
        ("(x,y)", "($x, $y)"),
        ("desc", "$name"),
    ]
    
    objectToPlot = figure(
        #y_range = (0,1),
        #x_range = (0,1),
        title = column,
        x_axis_label = 'Index[i]',
        y_axis_label = column,
        plot_width = 1600,
        #plot_height = 900,
        sizing_mode='scale_width',
        tools=[HoverTool()],
        tooltips=TOOLTIPS,
        #tooltips="Data point @x for i has the value @y",
    )
    
    #dfPre.fillna(method = "bfill", inplace = True)
    #dfPre.fillna(method = "ffill", inplace = True)
    
    mean = dfPre.mean(axis=1)
    std = dfPre.std(axis=1)
    n = dfPre.count(axis=1)
    
    dfPre['mean'] = mean
    dfPre['std'] = std
    dfPre['n'] = n
    
    mean = dfInstruction.mean(axis=1)
    std = dfInstruction.std(axis=1)
    n = dfInstruction.count(axis=1)
    
    dfInstruction['mean'] = mean
    dfInstruction['std'] = std
    dfInstruction['n'] = n
    
    mean = dfPost.mean(axis=1)
    std = dfPost.std(axis=1)
    n = dfPost.count(axis=1)
    
    dfPost['mean'] = mean
    dfPost['std'] = std
    dfPost['n'] = n
    
    
    for colNAme in columnNames:
        
        
        
        objectToPlot.line(dfPre[colNAme].index.tolist(), dfPre[colNAme], name = f"Participant : {colNAme}", 
                          legend_label= str(colNAme),
                          line_color="green",
                          line_width = .3,
                          line_alpha = .6,
                         )
        objectToPlot.line(dfInstruction[colNAme].index.tolist(), dfInstruction[colNAme], name = f"Participant : {colNAme}", 
                          line_color="red",
                          line_width = .3,
                          line_alpha = .6)
        objectToPlot.line(dfPost[colNAme].index.tolist(), dfPost[colNAme], name = f"Participant : {colNAme}", 
                          line_color="blue",
                          line_width = .3,
                          line_alpha = .6)
    
    objectToPlot.line(dfPre['mean'].index.tolist(), dfPre['mean'], name = f"{column} mean at physical environment",
                      line_color="green",
                      line_width = 3,
                      line_alpha = .8)
    objectToPlot.line(dfInstruction['mean'].index.tolist(), dfInstruction['mean'], name = f"{column} mean at instruction phase environment",
                      line_color="red",
                      line_width = 3,
                      line_alpha = .8)
    objectToPlot.line(dfPost['mean'].index.tolist(), dfPost['mean'], name = f"{column} mean at break phase environment",
                      line_color="blue",
                      line_width = 3,
                      line_alpha = .8)
    
    
    objectToPlot.line(dfPre['mean'].index.tolist(), dfPre['mean'] + dfPre['std'], name = "SD",
                      line_dash = 'dotted',
                      line_color="green",
                      line_width = 3,
                      line_alpha = .8)
    data = dfInstruction['mean'] + dfInstruction['std']
    objectToPlot.line(dfInstruction['mean'].index.tolist(),data , name = "SD",
                      line_dash = 'dotted',
                      line_color="red",
                      line_width = 3,
                      line_alpha = .8)
    objectToPlot.line(dfPost['mean'].index.tolist(), dfPost['mean'] + dfPost['std'], name = "SD",
                      line_dash = 'dotted',
                      line_color="blue",
                      line_width = 3,
                      line_alpha = .8)
    
    objectToPlot.line(dfPre['mean'].index.tolist(), dfPre['mean'] - dfPre['std'], name = "SD",
                      line_dash = 'dotted',
                      line_color="green",
                      line_width = 3,
                      line_alpha = .8)
    objectToPlot.line(dfInstruction['mean'].index.tolist(), dfInstruction['mean'] - dfInstruction['std'], name = "SD",
                      line_dash = 'dotted',
                      line_color="red",
                      line_width = 3,
                      line_alpha = .8)
    objectToPlot.line(dfPost['mean'].index.tolist(), dfPost['mean'] - dfPost['std'], name = "SD",
                      line_dash = 'dotted',
                      line_color="blue",
                      line_width = 3,
                      line_alpha = .8)
    
    objectToPlot.background_fill_color = "beige"
    objectToPlot.background_fill_alpha = 0.5
    save(objectToPlot)
    

In [ ]:
drawThis("bpm")

In [ ]:
for column in columnNames:
    try:
        print(column)
        drawThis(column)
    except Exception as e:
        print("can't draw this!")
        print(e)
    continue

In [ ]:
def drawThisOLD(column):
    from bokeh.models.tools import HoverTool

    print(f"<+>==============Drawing={column}+++++++++++++++++++++++<<<")
    print(noTailsDataSet[1].head())
    print(pre[1])
    print("==============")
    pre = [noTailsDataSet[i].loc[:breakStarts[i]-1, column] for i in range(0, len(noTailsDataSet))]
    instruction = [noTailsDataSet[i].loc[audioStarts[i]:audioEnds[i]-1, column] for i in range(0, len(noTailsDataSet))]
    #print(instruction[1])
    post = [noTailsDataSet[i].loc[audioEnds[i]:breakEnds[i], column] for i in range(0, len(noTailsDataSet))]
    #print(post[1])
    #postAll = [noTailsDataSet[i].loc[:breakEnds[i], column] for i in range(0, len(noTailsDataSet))]

    #MEANS not needed now
    """
    meanForTS = lambda timeSeries: sum(timeSeries)/len(timeSeries)
    meansPre = [meanForTS(item) for item in pre]
    meansPost = [meanForTS(item) for item in post]

    print(f"{column} : {stats.shapiro(meansPre)}")
    plt.hist(meansPre,20)
    plt.show()
    
    print(f"{column} : {stats.shapiro(meansPost)}")
    plt.hist(meansPost,20)
    plt.show()

    print(f"{column} : {stats.ttest_rel(meansPre, meansPost)}")
    print(f"{column} : {stats.wilcoxon(meansPre, meansPost)}")
    """
    
    ###OTher way through DF
    dfPre = pd.DataFrame(pre)
    #dfPre = pd.pivot_table(dfPre)
    #print(dfPre) 
    dfInstruction = pd.DataFrame(instruction)
    #print(dfInstruction)
    dfPost = pd.DataFrame(post)    
    #print(dfInstruction)
    
    graphPath = "Graphs\/"+ column + ".html";
    output_file(graphPath)
    
    TOOLTIPS = [
            ("index", "$i"),
            ("(x,y)", "($pre[i], $pre[i].index.tolist())"),
            ("desc", "@subjectNo"),
        ]
    
    objectToPlot = figure(
        #y_range = (0,1),
        #x_range = (0,1),
        title = column,
        x_axis_label = 'Index[i]',
        y_axis_label = column,
        plot_width = 1600,
        #plot_height = 900,
        sizing_mode='scale_width',
        tools=[HoverTool()],
        #tooltips="Data point @x for i has the value @y",
    )
    
    sumPre = pd.DataFrame()
    sumInstruction = pd.DataFrame()
    sumPost = pd.DataFrame()
    
    for i in range(0, len(noTailsDataSet)):
        
        if i == 0:
            sumPre['TS'] = pre[i]
            sumPre['Weights'] = 1
            
            sumInstruction['TS'] = instruction[i]
            sumInstruction['Weights'] = 1
            
            sumPost['TS'] = post[i]
            sumPost['Weights'] = 1
        
        else:
            sumPre['TS'].iloc[:len(pre[i])-1] += pre[i]
            sumPre['Weights'].iloc[:len(pre[i])-1] += 1
            
            sumInstruction['TS'].iloc[:len(instruction[i])-1] += instruction[i]
            sumInstruction['Weights'].iloc[:len(instruction[i])-1] += 1
            
            sumPost['TS'].iloc[:len(post[i])-1] += post[i]
            sumPost['Weights'].iloc[:len(post[i])-1] += 1
        
        
        objectToPlot.line(pre[i].index.tolist(), pre[i], 
                          #legend_label='Consensys_HR',
                          line_color="green",
                          line_width = .5,
                          line_alpha = .5,
                          )
        objectToPlot.line(instruction[i].index.tolist(), instruction[i], 
                          #legend_label='Consensys_HR',
                          line_color="red",
                          line_width = .5,
                          line_alpha = .5)
        objectToPlot.line(post[i].index.tolist(), post[i], 
                          #legend_label='Consensys_HR',
                          line_color="blue",
                          line_width = .5,
                          line_alpha = .5)
    # for mean:
    sumPre['TS'] = sumPre['TS'] / sumPre['Weights']
    sumInstruction['TS'] = sumInstruction['TS'] / sumInstruction['Weights']
    sumPost['TS'] = sumPost['TS'] / sumPost['Weights']
    
    objectToPlot.line(sumPre['TS'].index.tolist(), sumPre['TS'], 
                      #legend_label='Consensys_HR',
                      line_color="green",
                      #line_width = (sumPre['Weights'] + 1)/10,
                      line_width = 5,
                      line_alpha = .9)
    objectToPlot.line(sumInstruction['TS'].index.tolist(), sumInstruction['TS'], 
                      #legend_label='Consensys_HR',
                      line_color="red",
                      #line_width = (sumInstruction['Weights'] + 1)/10,
                      line_width = 5,
                      line_alpha = .9)
    objectToPlot.line(sumPost['TS'].index.tolist(), sumPost['TS'], 
                      #legend_label='Consensys_HR',
                      line_color="blue",
                      #line_width = (sumPost['Weights'] + 1)/10,
                      line_width = 5,
                      line_alpha = .9)
    
    #for i in range(0, len(noTailsDataSet)):
    
    objectToPlot.background_fill_color = "beige"
    objectToPlot.background_fill_alpha = 0.5
    save(objectToPlot)
    

In [ ]:
pre[:]

In [ ]:

graphPath = "Graphs\/"+ participantID + "_" + str(participantGR) + "-HR.html";
    output_file(graphPath)
    
    objectToPlot = figure(
        #y_range = (0,1),
        #x_range = (0,1),
        title = 'Heart Rate data - ID: ' + str(participantID) + ' GR: ' + str(participantGR),
        x_axis_label = 'Index[i]',
        y_axis_label = 'HR[BPM]',
        plot_width = 1600,
        #plot_height = 900,
        sizing_mode='scale_width'
    )
    
    objectToPlot.line(HP_data.index.tolist(), HP_data['HR_HeartPy_interpolated'], 
                      legend_label='HP_HR-interpolated', line_color="red", line_width = 1)
    objectToPlot.line(HP_data.index.tolist(), HP_data['bpm'], 
                      legend_label='HeartPy_HR', line_color="blue", line_width = 4,
                      line_dash = 'dotted', line_alpha = .7)
    objectToPlot.line(Consensys_data.index.tolist(), Consensys_data['PPG_to_HR_CONSENSYS'], 
                      legend_label='Consensys_HR', line_color="green", line_width = 4,
                      line_dash = 'dotted', line_alpha = .7)
    objectToPlot.line(Consensys_data.index.tolist(), Consensys_data['HR_CONSENSYS_interpolated'], 
                      legend_label='Consensys_HR-interpolated', line_color="green", line_width = 1)
    
    objectToPlot.background_fill_color = "beige"
    objectToPlot.background_fill_alpha = 0.5
    save(objectToPlot)
